# Swelling study

In [ ]:
import itertools as ittls
import sys
from pprint import pprint
from typing import Mapping, Any, List, Tuple, Union
from tqdm.notebook import tqdm

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import tri
import numpy as np
import scipy.signal as signal
import dolfin as dfn
import h5py

from femvf import forward, statefile as sf, meshutils
from femvf.models.dynamical import (solid as dsolid, fluid as dfluid)
from femvf.load import load_dynamical_fsi_model

from blockarray import blockvec

from vfsig import modal, fftutils, clinical

from exputils import h5utils
import main
import viciouscycle as vc

sys.path.append('../vf-onset-sensitivity')
import libhopf

dfn.set_log_level(50)

## Constants

In [ ]:
## Load the model being used
# Use one of `CLSCALE` and `DT` combos below to see the effects of refinement
# on trends
CLSCALE = 0.5

# CLSCALE = 0.5
# DT = 2.5e-5
ExpParam = main.ExpParam

In [ ]:
## Default parameter values
# (you may have to change these depending on the runs)
DT = 1.25e-5
TF = 0.5

DT = 5e-5
TF = 0.5

ECOV = 2.5e3 * 10
EBOD = 5e3 * 10
PSUB = 0.3e3 * 10
PSUB = 0.6e3 * 10

VCOVS = np.array([1, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3])
MCOVS = np.array([0, -0.4, -0.8, -1.2, -1.6])

VCOVS = np.array([1, 1.025, 1.05, 1.075, 1.1])
MCOVS = np.array([0.0])

OUT_DIR = 'out_debug_dt_psub'
OUT_DIR = 'out'
POST_DIR = OUT_DIR

DEFAULT_PARAM = ExpParam({
    'MeshName': 'M5_BC',
    'GA': 3.0,
    'clscale': 0.75,
    'DZ': 1.50, 'NZ': 15,
    'Ecov': ECOV,
    'Ebod': EBOD,
    'psub': PSUB,
    'vcov': 1.0,
    'mcov': -0.8,
    'dt': DT,
    'tf': TF,
    'ModifyEffect': '',
    'SwellingDistribution': 'field.tavg_viscous_rate',
    'SwellingModel': 'power'
})

## Function definitions

### Loading results

In [ ]:
## Functions for accessing post-processed results

DEFAULT_PARAM = ExpParam({
    'MeshName': 'M5_BC',
    'GA': 3.0,
    'clscale': 0.75,
    'DZ': 1.50, 'NZ': 15,
    'Ecov': ECOV,
    'Ebod': EBOD,
    'psub': PSUB,
    'vcov': 1.0,
    'mcov': -0.8,
    'dt': DT,
    'tf': TF,
    'ModifyEffect': '',
    'SwellingDistribution': 'field.tavg_viscous_rate',
    'SwellingModel': 'power'
})

def load_measures(
        data: Mapping[str, Any],
        measure_name: str,
        params: List[Mapping[str, Any]] | Mapping[str, Any],
        default_param: ExpParam=DEFAULT_PARAM
    ):
    """
    Return measures for each supplied parameter

    Parameters
    ----------
    data: Mapping[str, Any]
        A mapping from result names to their values
    measure_name: str
        A measure name to load
    params: List[Mapping[str, Any]]
        A mapping from parameter labels to parameter value(s). Measurements
        are extracted from these parameters.
    defaults:
        Default values for unspecified parameters
    """
    if isinstance(params, dict):
        vals = data[f'{default_param.substitute(params).to_str()}/{measure_name}']
    elif isinstance(params, list):
        vals = [
            data[f'{default_param.substitute(param).to_str()}/{measure_name}']
            for param in params
        ]
    elif isinstance(params, np.ndarray):
        vals = np.array([
            data[f'{default_param.substitute(param).to_str()}/{measure_name}']
            for param in params.flat
        ])
        vals = np.reshape(vals, (*params.shape, -1))
    else:
        raise TypeError(f'Invalid type for `params`, {type(params)}')

    return vals

def load_measures_np(
        data: Mapping[str, Any],
        measure_name: str,
        params: List[Mapping[str, Any]],
        default_param: ExpParam=DEFAULT_PARAM
    ):
    """
    Return measures for each supplied parameter(s) (see `load_measures`)

    This functions returns outputs in a numpy array
    """
    return np.array(load_measures(data, measure_name, params, default_param))


In [ ]:
# clscales = [2, 1, 0.5]
# clscales = [0.75]
# mesh_names = ['M5_BC']
# _params = [
#     DEFAULT_PARAM.substitute({'MeshName': name, 'clscale': clscale})
#     for name in mesh_names for clscale in clscales
# ]
# MODELS = {
#     main.setup_mesh_name(params): main.setup_model(params)
#     for params in _params
# }

class ModelLoader:
    """
    Load models from a cache
    """

    def __init__(self, models: Mapping[str, Any]=None):
        if models is None:
            models = {}
        self._models = models

    def __call__(self, param: ExpParam):
        mesh_name = main.setup_mesh_name(param)

        if mesh_name not in self._models:
            self._models[mesh_name] = main.setup_model(param)

        return self._models[mesh_name]

load_model = ModelLoader()

In [ ]:
def normal(x):
    """
    Return the normal vector to a 2D curve `x`
    """
    assert x.ndim == 2
    assert x.shape[-1] == 2

    dx = np.zeros(x.shape)
    dx[1:, :] = x[1:] - x[:-1]
    s = np.cumsum(np.linalg.norm(dx, axis=-1))

    # Tangent
    dx = np.gradient(x[:, 0], s, axis=0)
    dy = np.gradient(x[:, 1], s, axis=0)

    # Non-unit normal
    nx = dy
    ny = -dx
    n = np.stack([nx, ny], axis=-1)
    return n/np.linalg.norm(n, axis=-1, keepdims=True)

def offset_normal(x, n):
    return x + n

### Processing time signals

In [ ]:
def tavg(y, t, axis=-1):
    """
    Return the time average of a signal
    """
    tavg_y = np.trapz(y, x=t, axis=axis)/np.trapz(np.ones(t.shape), x=t, axis=axis)
    return tavg_y

In [ ]:
## Setup function to compute SPL from t/q

def proc_prms(t, q):
    """
    Return the RMS radiated pressure at 1m using a piston-in-baffle approximation
    """
    # t = DATA[f'{case_name}/time']
    # q = DATA[f'{case_name}/q']
    dt = t[1]-t[0]

    # Assume a 2cm vocal fold length
    # This is needed to get a true flow rate since the 1D model flow rate
    # is over one dimension
    piston_params = {
        'r': 100.0,
        'theta': 0.0,
        'a': 1.0,
        'rho': 0.001225,
        'c': 343*100
    }
    VF_LENGTH = 2
    win = signal.windows.tukey(q.size, alpha=0.15)
    wq = win*q*VF_LENGTH
    fwq = np.fft.rfft(wq)
    freq = np.fft.rfftfreq(wq.size, d=dt)

    fwp = clinical.prad_piston(fwq, f=freq*2*np.pi, piston_params=piston_params)

    # This computes the squared sound pressure
    psqr = fftutils.power_from_rfft(fwp, fwp, n=fwq.size)

    # The rms pressure in units of Pa
    prms = np.sqrt(psqr/fwp.size)

    return prms

### Processing the model

In [ ]:
def make_solve_static(model):
    """
    Return the static state for a solid model with no external loading
    """
    forms = model.solid.forms
    f1uva = forms['form.un.f1uva']
    u1 = forms['coeff.state.u1']
    bc_dir = forms['bc.dirichlet']

    def solve_static():
        dfn.solve(f1uva==0, u1, bcs=[bc_dir])
        x = model.state1.copy()
        x['v'][:] = 0
        x['a'][:] = 0
        return x
    return solve_static

In [ ]:
def get_dx(model, name=None):
    cell_label_to_id = model.solid.residual.mesh_function_label_to_value('cell')
    _dx = model.solid.residual.measure('dx')
    if name is None:
        dx = _dx
    elif name == 'cover':
        # dx = (
        #     _dx(int(cell_label_to_id['inferior']))
        #     +_dx(int(cell_label_to_id['medial']))
        #     + _dx(int(cell_label_to_id['superior']))
        # )
        dx = _dx(int(cell_label_to_id['cover']))
    else:
        dx = _dx(int(cell_label_to_id[name]))
    return dx

### For plotting

In [ ]:
## Functions for plotting percent change
def per_change_forward(y, y0):
    return (y-y0)/y0 * 100

def per_change_inv(y, y0):
    return y*y0/100 + y0

In [ ]:
def annotate_vertical_trend(ax, text, x, y1, y2):
    """
    Annotate a trend from 'y1' to 'y2'
    """
    ax.annotate(
        text, (x, y2), xytext=(x, y1),  ha='center',
        textcoords='data', xycoords='data',
        arrowprops={'arrowstyle': '->'}
    )

In [ ]:
def plot_tripcolor_seq(axs, trian, zdata, zmin=None, zmax=None):
    """
    Plot a sequence of `tripcolor` data with shared z limits

    By default, all plots are normalized between the min/max of all z data.

    Parameters
    ----------
    axs : mpl.axes.Axes
        An array of axes to plot in
    trian : mpl.triangulation.Triangulation
        The triangulation used for data
    zdata : List[np.ndarray]
        A list of z data for each tri-plot

    Returns
    -------
    List[mpl.artist.Artist]
        A list of artists for each `tripcolor` call. You can use any of the
        artists to create a colorbar since they have the same normalization.
    """
    if zmin is None:
        zmin = np.min(zdata)
    if zmax is None:
        zmax = np.max(zdata)
    kwargs = {'vmin': zmin, 'vmax': zmax, 'edgecolors': 'none'}

    fspace_cg1 = dfn.FunctionSpace(model.solid.residual.mesh(), 'CG', 1)
    fspace_dg0 = dfn.FunctionSpace(model.solid.residual.mesh(), 'DG', 0)
    z_cg1 = dfn.Function(fspace_cg1)
    z_dg0 = dfn.Function(fspace_dg0)

    zdata_flat = zdata.reshape(-1, zdata.shape[-1])
    for z, ax in zip(zdata_flat, axs.flat):
        artists = ax.tripcolor(trian, z, **kwargs)
        # ax.tricontour(trian, y, [0])

        ## Project the dg0 stress to CG1 so you can plot contours
        # print(y_dg0.vector().size(), y.shape)
        z_dg0.vector()[:] = z
        z_vertex = dfn.project(z_dg0, fspace_cg1, function=z_cg1).vector()[VERT_TO_SDOF]
        # Setting 'Greys' cmap and vmin/vmax appropriately will ensure a specific color for the contour line
        # cmap = plt.get_cmap(name='Greys')
        ax.tricontour(trian, z_vertex, [0], colors=['k'], linewidths=[1])
    return artists

## Debug: Inspect individual case

In [ ]:
with sf.StateFile(MODEL, 'SwellingStep0.h5', mode='r') as f:
    print(f)
    q = np.array([
        np.sum([
            f.get_state(ii)[f'fluid{n}.q'][0]
            for n in range(len(f.model.fluids))
        ])
        for ii in range(f.size)
    ])
    t = f.get_times()


In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(q)
print(q)
# [   0.         3642.94386451 3632.68382534 ... 1383.11511316 1408.29864461
# 1443.04533938]
fund_freq, fund_phase, dfreq, dphase, info = \
        modal.fundamental_mode_from_peaks(q, t[1]-t[0], height=np.max(q)*0.8)

print(fund_freq)

In [ ]:
psub = 600*10
param = DEFAULT_PARAM.substitute({
    'psub': psub,
    'clscale': 0.75,
    'vcov': 1.1, 'mcov': 0,
    'dt': 5e-5, 'tf': 0.25,
    'SwellingDistribution': 'field.tavg_viscous_rate',
    'SwellingModel': 'linear'
})

In [ ]:
keys = list(DATA.keys())
result_names = set(key.split('/')[-1] for key in keys)
case_names = set(key.split('/')[0] for key in keys)

print(case_names)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 2))

t = DATA[param.to_str()+'/time.t']
y = DATA[param.to_str()+'/time.spatial_stats_vm']
y = DATA[param.to_str()+'/time.gw']
ax.plot(t, y)

ax.set_xlim(0.125, 0.25)

## Prototyping

### Plot variations in glottal waveform with $p_\mathrm{sub}$ 

In [ ]:
param = main.ExpParam({
    'MeshName': 'M5_BC', 'clscale': 0.94,
    'GA': 3,
    'DZ': 1.5, 'NZ': 12,
    'Ecov': 2.5e4, 'Ebod': 5e4,
    'vcov': 1, 'mcov': 0.0,
    'psub': 600*10,
    'dt': 5e-5, 'tf': 0.50,
    'ModifyEffect': '',
    'SwellingDistribution': 'uniform',
    'SwellingModel': 'power'
})
model = load_model(param)

In [ ]:
ini_state, control, prop = main.setup_state_control_prop(param, model)

In [ ]:
dpsub = 100
psubs = dpsub*10*np.arange(1, int(800//dpsub)+0.5)

In [ ]:
controls = []
for psub in psubs:
    _control = control[0].copy()
    for n in range(len(model.fluids)):
        _control[f'fluid{n}.psub'] = psub
    controls.append(_control)

times = 5e-5*np.arange(2**11)

In [ ]:
for n, control_n in tqdm(enumerate(controls), total=len(controls)):
    with sf.StateFile(model, f'psub_trend{n}.h5', mode='w') as f:
        forward.integrate(model, f, ini_state, [control_n], prop, times)

In [ ]:
qs = []

fpaths = [f'SwellingStep{n}.h5' for n in range(len(psubs))]
for n, fpath in enumerate(fpaths):
    with sf.StateFile(model, f'psub_trend{n}.h5', mode='r') as f:
        q = main.proc_glottal_flow_rate(f)
        qs.append(q)

In [ ]:
fig, ax = plt.subplots(1, 1)

for psub, q in zip(psubs, qs):
    ax.plot(times, q, label=f"{psub/10:.1f} Pa")

ax.set_xlabel(r"$t$ $[\mathrm{s}]$")
ax.set_ylabel(r"$q$ $[\mathrm{cm^2/s}]$")

ax.legend()

### Plot vicious cycle

In [ ]:
model = load_model(DEFAULT_PARAM.substitute({'clscale': 0.94, 'NZ': 12}))

In [ ]:
num_vc_step = 7

n = 1

spls = []
psubs = []
qs = []
dmg_rates = [] 
vfields = []

for n in tqdm(range(num_vc_step)):
    with sf.StateFile(
            model, f'out/vicious_cycle/SwellingStep{n}.h5', mode='r'
        ) as f:
        # Find subglottal pressure
        psub = f.get_control(0).sub[f'fluid5.psub'][0]

        t = f.get_times()

        # Process the flow rate by a weighted average over each coronal cross-section
        num_fluid = len(model.fluids)
        q = main.proc_glottal_flow_rate(f)

        dmg_rate = vc.proc_damage_rate(model, f)

        vfield = f.get_prop()['v_swelling'][:]

    weights = np.ones(num_fluid)
    weights[[0, -1]] = 0.5
    # q = np.sum(np.array(qs)[..., 0] * weights[:, None], axis=0)/np.sum(weights)

    # Find SPL (RMS radiated pressure)
    n = int(q.size//2)
    prms = proc_prms(t[n:], q[n:])/10
    spl = 20 * np.log10(prms/2e-5)
    qs.append(q)
    spls.append(spl)
    psubs.append(psub)
    dmg_rates.append(dmg_rate)
    vfields.append(np.array(vfield))

In [ ]:
vtotals = []
mesh = model.solid.residual.mesh()
_fspace = dfn.FunctionSpace(mesh, 'DG', 0)
_vfield = dfn.Function(_fspace)
dx = dfn.Measure('dx', mesh)
for vfield in vfields:
    _vfield.vector()[:] = vfield
    vtotal = dfn.assemble(_vfield*dx)
    vtotals.append(vtotal)

In [ ]:
# Compute a dimensionless time based on a reference damage rate
dmg_rate_ref = 1e5
time_steps = np.array([0] + [
    np.max(dmg_rate)/dmg_rate_ref 
    for dmg_rate in dmg_rates[:-1]
])
times = np.cumsum(time_steps)

print(times)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(FIG_LX_WIDE/2, FIG_LY), sharex=True)

vc_steps = np.arange(num_vc_step)

axs[0].plot(times, np.array(psubs)/10)
axs[0].set_ylabel(r"$p_\mathrm{sub}$ $[\mathrm{Pa}]$")

axs[1].plot(times, np.array(vtotals)/vtotals[0])
axs[1].set_ylabel(r"$V/V_0$ $[\%]$")

# axs[2].plot(times, np.array(spls)-spls[0])
# axs[2].set_ylabel(r"$\Delta\mathrm{SPL}$ $[\mathrm{dB}]$")

axs[-1].set_xticks(times)
axs[-1].set_xlabel("Vicious cycle time")

fig.savefig(f'{FIG_DIR}/ViciousCycle--PsubVolume.svg')

In [ ]:
# np.bool

from pyvista import examples
uni = examples.load_uniform()

pl = pv.Plotter(shape=(1, 2), border=False)
pl.add_mesh(uni, show_edges=True)
# pl.subplot(0, 1)
# pl.add_mesh(uni, scalars='Spatial Cell Data', show_edges=True)
pl.show()

In [ ]:
import pyvista

from pyvista import demos
pyvista.start_xvfb()
demos.plot_wave()

### Plot coronal cross-sections

In [ ]:
### Try to generate coronal cross section meshes given you know all the facet indices on the cross section

In [ ]:
from femvf.meshutils import extract_zplane_facets

In [ ]:
model = load_model(DEFAULT_PARAM)

In [ ]:
print(DEFAULT_PARAM)

In [ ]:
mesh = model.solid.residual.mesh()
nz = DEFAULT_PARAM['NZ']
facets = extract_zplane_facets(mesh, 1.5/nz)
facets = extract_zplane_facets(mesh, 0.0)
facets_idx = [facet.index() for facet in facets]

In [ ]:
mesh.geometry()

In [ ]:
facets_idx = [1, 100]
cells = mesh.cells()[facets_idx]
cells = mesh.facets()
coords = mesh.coordinates()
print(cells.shape)

fig, ax = plt.subplots(1, 1)
# ax.tripcolor(*coords[:, :2].T, np.ones(cells.shape[:1]), triangles=cells)
ax.triplot(*coords[:, :2].T, triangles=cells)

In [ ]:
mesh.cells()[facets_idx]
# mesh
# mesh

## Load post-processed results

In [ ]:
post_dir = 'out'

# This dictionary stores all the post-processed data
DATA = {}

In [ ]:
## Load the main results
with h5py.File(f'{post_dir}/postprocess.h5', mode='r') as f:
    DATA = h5utils.h5_to_dict(f, DATA)

## Load the independence results
# with h5py.File(f'{post_dir}/independence/postprocess.h5', mode='r') as f:
#     DATA = h5utils.h5_to_dict(f, DATA)

### Summary of post-processed quantities

In [ ]:
# Get a text summary of what cases were run and what measures were processed
from pprint import pprint
case_names = [key.split('/')[0] for key in DATA.keys() if 'SwellingModel' in key]
case_names = list(set(case_names))
# pprint(case_names)

meas_names = [key.split('/')[1] for key in DATA.keys()]
meas_names = list(set(meas_names))
# pprint(meas_names)

In [ ]:
meas_names

### Additional post-processing

In [ ]:
## Post-process stress fields at fixed points
from femvf import static
from femvf.postprocess.solid import StressVonMisesField, StressHydrostaticField

# Loop through each simulation and solve for the fixed point
for case_name in tqdm(case_names):
    # Load the corresponding parameters and appropriate model
    param = main.ExpParam(case_name)
    model = load_model(param)

    svm_field = StressVonMisesField(model)
    shydro_field = StressHydrostaticField(model)

    state, controls, prop = main.setup_state_control_prop(param, model)
    control = controls[0]
    model.set_prop(prop)
    model.set_ini_state(state)
    model.dt = 1e6

#     state_fp = main.setup_ini_state(param, model, dvcov=0.1)
#     svm = svm_field(state_fp, control, prop)
#     shydro = shydro_field(state_fp, control, prop)

#     # Save the fixed-point von Mises/hydrostatic stresses
#     DATA[f'{case_name}/field_fp_vm'] = svm
#     DATA[f'{case_name}/field_fp_hydrostatic'] = shydro

In [ ]:
import ufl
_sig_update = {}

for case_name in tqdm(case_names):
    param = main.ExpParam(case_name)
    model = load_model(param)

    dx = get_dx(model)
    dx_cover = get_dx(model, 'cover')
    _f = dfn.Function(model.solid.residual.form['coeff.prop.emod'].function_space())
    # dofs_cover = meshutils.

    ## Add region averaged measures of stress fields
    stress_field_keys = {'field_tini_vm', 'field_tavg_vm', 'field_fp_vm'}
    stress_field_keys = {'field_tini_vm', 'field_tavg_vm', 'field_fp_vm'}
    for stress_field_key in stress_field_keys:
        key = f'{case_name}/{stress_field_key}'
        if key in DATA:
            # print(DATA[key].shape)
            try:
                _f.vector()[:] = DATA[key]
                cover_vol = dfn.assemble(1*dx_cover)
                _sig_update[f'{case_name}/cover_avg_{stress_field_key}'] = dfn.assemble(_f*dx_cover)/cover_vol
                # _sig_update[f'{case_name}/cover_max_{stress_field_key}'] = np.max(_f.vector()[dofs_cover])
                # _sig_update[f'{case_name}/medial_avg_{stress_field_key}'] = dfn.assemble(_f*dx_medial)/medial_vol
                # _sig_update[f'{case_name}/medial_max_{stress_field_key}'] = np.max(_f.vector()[dofs_medial])
            except:
                pass

    ## Add time/space averaged viscous dissipation rate
    t = DATA[f'{case_name}/time.t']
    N = t.size
    wvisc_stats = DATA[f'{case_name}/time.spatial_stats_viscous']
    _sig_update[f'{case_name}/avg_wvisc'] = tavg(wvisc_stats['avg'][-N//2:], t[-N//2:])

    ## Add cover mass and volume
    state_swollen = main.setup_ini_state(param, model, dvcov=0.5)
    model.set_fin_state(state_swollen)

    # with sf.StateFile(model, f'{POST_DIR}/{case_name}.h5', mode='r') as f:
    #     model.set_fin_state(f.get_state(0))
    #     model.set_prop(f.get_prop())

    dens = model.solid.residual.form['coeff.prop.rho']
    disp = model.solid.residual.form['coeff.state.u1']
    def_grad = ufl.grad(disp)
    j = ufl.det(def_grad + dfn.Identity(3))
    _sig_update[f'{case_name}/mass_cover'] = dfn.assemble(dens*dx_cover)
    _sig_update[f'{case_name}/mass_total'] = dfn.assemble(dens*dx)
    _sig_update[f'{case_name}/vol_cover'] = dfn.assemble(j*dx_cover)
    _sig_update[f'{case_name}/vol_total'] = dfn.assemble(j*dx)

DATA.update(_sig_update)

In [ ]:
## Add SPL and fundamental frequency to results
for case_name in tqdm(case_names):
    ## Add Fo
    t = DATA[f'{case_name}/time.t']
    gw = DATA[f'{case_name}/time.gw']
    N = gw.size//2
    gw = gw[-N:]
    dt = t[1]-t[0]

    fund_freq, fund_phase, dfreq, dphase, info = \
        modal.fundamental_mode_from_peaks(gw, dt, height=np.max(gw)*0.8)
    # print(fund_freq)
    _sig_update[f'{case_name}/fund_freq'] = fund_freq
    _sig_update[f'{case_name}/dfreq'] = dfreq
    _sig_update[f'{case_name}/period_sample'] = 1/(fund_freq *dt)

    fund_freq, fund_phase, dfreq, dphase, info = \
        modal.fundamental_mode_from_rfft(gw, t[1]-t[0])
    # print(case_name, fund_freq)
    # if fund_freq == np.NaN:
    #     print(case_name)
    _sig_update[f'{case_name}/fund_freq_dft'] = fund_freq
    _sig_update[f'{case_name}/dfreq_dft'] = dfreq

    ## Add SPL
    t = DATA[f'{case_name}/time.t']
    _q = DATA[f'{case_name}/time.q']
    
    # NOTE: This is done because 'time.q' wasn't computed with an average
    # across all fluid slices before!
    param = ExpParam(case_name)
    q = _q/(param['NZ'])

    # Divide `prms` by 10 to get units of [Pa]
    prms = proc_prms(t, q)/10
    _sig_update[f'{case_name}/prms'] = prms
    _sig_update[f'{case_name}/spl'] = 20*np.log10(prms/20e-6)

    ## Add vibration amplitude
    # assert gw.ndim == 1
    _sig_update[f'{case_name}/amplitude'] = gw.max()-gw.min()

DATA.update(_sig_update)

## Figures

### Figure constants

Converts the output figure names to a manuscript ordered names (Fig1, Fig2, ...)

In [ ]:
fig_names = [
    'SwelledGeometryVsV',
    'CoverMassVsV',
    'F0VsV',
    'ContactVMCycleDynamicsVsV',
    'StressFieldvsV',
    'AvgVMStressVsV'
]

In [ ]:
## Set any figure constants
FIG_TYPE = 'manuscript'
FIG_TYPE = 'presentation'

CM = 1/2.54
if FIG_TYPE == 'manuscript':
    FIG_DIR = 'fig/manuscript'
    FIG_EXT = 'pdf'

    FIG_LX = 8.4*CM
    FIG_LX_WIDE = 17.4*CM

    FIG_LY = 23.4*CM
    FONTSIZE = 9
elif FIG_TYPE == 'presentation':
    FIG_DIR = 'fig/presentation'
    FIG_EXT = 'svg'

    FIG_LX = 7.5*CM
    FIG_LX_WIDE = 30*CM

    FIG_LY = 13*CM
    FONTSIZE = 12

SIZE_SMALL = (FIG_LX, FIG_LX)
SIZE_MED = (FIG_LX, 1.25*FIG_LX)
SIZE_MED_WIDE = (FIG_LX_WIDE, 1.25*FIG_LX)
SIZE_LARGE = (FIG_LX, 1.5*FIG_LX)
SIZE_LARGE_WIDE = (FIG_LX_WIDE, 1.5*FIG_LX)

DPI = 250

# Colors
COLOR_CYCLE = plt.rcParams['axes.prop_cycle'].by_key()['color']
MARKER_CYCLE = ['.', '*', 'x', 'o']
LS_CYCLE = ['-', '-.', ':', '--', (0, (1, 4))]

mpl.rcParams.update({
    'figure.dpi': 200,
    'font.size': FONTSIZE,
    'text.usetex': True,
    'legend.handlelength': 2,
    'legend.frameon': False,
    'text.latex.preamble': (
        r'\usepackage{amsmath}'
        r'\usepackage{siunitx}'
        r'\usepackage{sansmath}'
        r'\sansmath'
        r'\sisetup{detect-all}'
        r'\newcommand{\avg}[1]{\underset{#1}{\mathrm{avg}} \, }'
    )
})
# mpl.rcParams['']

### Mesh for Inkscape/schematic

In [ ]:
model = load_model(DEFAULT_PARAM)

mesh = model.solid.residual.mesh()
fspace_CG1 = dfn.VectorFunctionSpace(mesh, 'CG', 1)
xref = fspace_CG1.tabulate_dof_coordinates()[::3]
# xref.shape
# vert_to_vdof

In [ ]:
# Plot mesh for Inkscape
fig, ax = plt.subplots(1, 1, figsize=(8.4*CM, 8.4*CM))

ax.set_aspect(1)

xy_vert = np.array(XREF)[VERT_TO_VDOF].reshape(-1, 2)
print(xy_vert.shape)
ax.triplot(*xy_vert.T, CELLS, lw=0.5)

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/Mesh.svg')

### Onset swelling plots

In [ ]:
DYN_MODEL = load_dynamical_fsi_model(
    mesh_path,
    None,
    SolidType=dsolid.SwellingKelvinVoigtWEpitheliumNoShape,
    FluidType=dfluid.BernoulliAreaRatioSep
)
LDYN_MODEL = load_dynamical_fsi_model(
    mesh_path,
    None,
    SolidType=dsolid.SwellingKelvinVoigtWEpitheliumNoShape,
    FluidType=dfluid.LinearizedBernoulliAreaRatioSep
)
HOPF_MODEL = libhopf.HopfModel(DYN_MODEL, LDYN_MODEL)

In [ ]:
%debug
## Set basic model parameters

# This sets the properties based on the 'main.py' script that runs the sims.
params_dict = DEFAULT_PARAM.data.copy()
params_dict['vcov'] = 1.1
params_dict['mcov'] = 0
params = ExpParam(params_dict)
prop = main.set_basic_props(DYN_MODEL, params)
# prop.print_summary()

# Manually change properties you want to check here
mesh_coords = DYN_MODEL.solid.forms['mesh.mesh'].coordinates()
ymax = mesh_coords[:, 1].max()
ygap = 0.03
prop['ycontact'] = ymax + 9/10*ygap
prop['ymid'] = ymax + ygap
prop['area_lb'] = 2*1/10*ygap
# prop['nu'] = 0.4999
# prop['emod_membrane'] = 0e3*10
# prop['emod'][cells_cover] = 2.5e3*10

# prop.print_summary()

In [ ]:
## Solve for the displacement with swelling
MODEL.set_prop(prop)
nload = 25

# MODEL.solid.control.print_summary()

state_fp, solve_info = main.solve_static_swollen_config(
    MODEL.solid, MODEL.solid.control, MODEL.solid.prop, nload
)
print(solve_info)
state0 = MODEL.solid.state0.copy()
state0[:] = 0

def assem_res():
    res = dfn.assemble(MODEL.solid.forms['form.un.f1uva'])
    MODEL.solid.forms['bc.dirichlet'].apply(res)
    return res

MODEL.solid.set_fin_state(state0)
print(assem_res().norm('l2'))
MODEL.solid.set_fin_state(state_fp)
print(assem_res().norm('l2'))

statefp_0 = DYN_MODEL.state.copy()
statefp_0[:] = 0
statefp_0['u'] = state_fp['u']

In [ ]:
def solve_fp_r(res, control, prop):
    # Generate a swollen deformation as the initial guess for a fixed-point
    res.set_prop(prop)
    sl_control = res.solid.control
    sl_control[:] = 0

    nload = int(20*np.max(prop['v_swelling'])/1.3)
    sl_state_swollen, solve_info = main.solve_static_swollen_config(
        res.solid, sl_control, res.solid.prop, nload
    )
    if solve_info['status'] == -1:
        raise RuntimeError(
            "Static swollen deformation could not be computed with info: "
            f"{solve_info}"
        )

    # Use the static swollen state as the initial guess for solving the
    # fixed-point
    xfp_0 = res.state.copy()
    xfp_0[:] = 0
    xfp_0['u'] = sl_state_swollen['u']
    xfp_n, solve_info = libhopf.solve_fp(res, control, prop, xfp_0=xfp_0, psub_incr=1e4)
    if solve_info['status'] == -1:
        raise RuntimeError(
            "Fixed point could not be computed with info: "
            f"{solve_info}"
        )

    return xfp_n, solve_info

In [ ]:
# Quick test to see if `solve_fp_r` works
control = DYN_MODEL.control.copy()
control['psub'] = 400*10
control['psup'] = 0
solve_fp_r(DYN_MODEL, control, prop)

#### Plot growth rate vs psub

In [ ]:
## Compute the growth rate vs. psub
psubs = np.arange(10, 501, 100)*10
control = DYN_MODEL.control.copy()
def set_control(control, psub):
    control['psub'] = psub
    return control
gammas = np.array([
    libhopf.solve_least_stable_mode(
        DYN_MODEL,
        solve_fp_r(DYN_MODEL, set_control(control, psub), prop)[0],
        set_control(control, psub),
        prop
    )[0]
    for psub in tqdm(psubs)
])

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(psubs/10, gammas.real)
ax.axhline(0, ls='-.')
ax_twin = ax.twinx()
ax_twin.plot(psubs/10, gammas.imag/2/np.pi, color=COLOR_CYCLE[1])

ax.set_ylabel("$\omega_\mathrm{real}$", color=COLOR_CYCLE[0])
ax_twin.set_ylabel("$\omega_\mathrm{imag}$ [Hz]", color=COLOR_CYCLE[1])
ax.set_xlabel("$p_\mathrm{sub}$ [Pa]")

#### Onset pressure trend with swelling

In [ ]:
def hopf_from_v(vv, gap_compensation=False):
    params_dict = DEFAULT_PARAM.data.copy()
    params_dict['vcov'] = vv
    params_dict['mcov'] = 0
    params = ExpParam(params_dict)
    prop = main.set_basic_props(DYN_MODEL, params)

    # Compute `ymax` when accounting for swelling induced prephonatory gap changes
    DYN_MODEL.set_prop(prop)
    control = DYN_MODEL.solid.control
    control[:] = 0
    state_swollen, info = main.solve_static_swollen_config(
        DYN_MODEL.solid, control, DYN_MODEL.solid.prop, nload=10
    )
    if gap_compensation:
        ymax = (DYN_MODEL.solid.XREF + state_swollen.sub['u'])[1::2].max()
    else:
        ymax = DYN_MODEL.solid.XREF[1::2].max()

    # Manually change properties here
    ygap = 0.03
    prop['ycontact'] = ymax + ygap - 1/10*ygap
    prop['ymid'] = ymax + ygap
    prop['area_lb'] = 2*1/10*ygap

    psubs = np.arange(10, 301, 100)*10
    xhopf_0 = HOPF_MODEL.state.copy()
    xhopf_0[:] = libhopf.gen_xhopf_0(
        HOPF_MODEL.res, prop, HOPF_MODEL.E_MODE, psubs, solve_fp_r=solve_fp_r
    )

    xhopf, info = libhopf.solve_hopf_by_newton(HOPF_MODEL, xhopf_0, prop)
    return xhopf

def ponset_from_v(vv, gap_compensation=False):
    xhopf = hopf_from_v(vv, gap_compensation)
    return xhopf.sub['psub'][0]

In [ ]:
# %debug
vvs = np.array([1.0, 1.1, 1.2, 1.3])

# vvs = np.array([1.3])
xhopfs_ngap = [hopf_from_v(vv, False) for vv in tqdm(vvs)]
xhopfs_ygap = [hopf_from_v(vv, True) for vv in tqdm(vvs)]

In [ ]:
ponsets_ngap = np.array([xhopf.sub['psub'] for xhopf in xhopfs_ngap])
ponsets_ygap = np.array([xhopf.sub['psub'] for xhopf in xhopfs_ygap])

fonsets_ngap = np.array([xhopf.sub['omega'] for xhopf in xhopfs_ngap])/(2*np.pi)
fonsets_ygap = np.array([xhopf.sub['omega'] for xhopf in xhopfs_ygap])/(2*np.pi)

In [ ]:
DEFAULT_PARAM

In [ ]:
if FIG_TYPE == 'presentation':
    figsize = (FIG_LX_WIDE, FIG_LY)
else:
    figsize = (FIG_LX, 0.25*FIG_LY)
fig, axs = plt.subplots(1, 2, figsize=figsize)

ax_p = axs[0]
ax_p.plot(vvs, ponsets_ngap/10, label="$\\bar{{m}}'=-0.8$")

ax_f = axs[1]
ax_f.plot(vvs, fonsets_ngap, label="$\\bar{{m}}'=-0.8$")

ax_p.set_xlabel("$v$")
ax_f.set_xlabel("$v$")
ax_p.set_ylabel("$p_\mathrm{on}$ [Pa]")
ax_f.set_ylabel("$f_\mathrm{on}$ [Hz]")
ax_p.legend()
# for ax in axs.flat:
#     ax.legend()

# ax_p.xaxis.set_tick_params(labelbottom=False)

fig.tight_layout()

fig.savefig(f'{FIG_DIR}/OnsetPressureVsSwellingPresentation.{FIG_EXT}')

In [ ]:
if FIG_TYPE == 'presentation':
    figsize = (FIG_LX_WIDE, FIG_LY)
else:
    figsize = (FIG_LX, 0.25*FIG_LY)
fig, axs = plt.subplots(1, 2, figsize=figsize)

ax_p = axs[0]
ax_p.plot(vvs, ponsets_ngap/10, label="variable prephonatory gap")
ax_p.plot(vvs, ponsets_ygap/10, label="constant prephonatory gap")

ax_f = axs[1]
ax_f.plot(vvs, fonsets_ngap, label="variable prephonatory gap")
ax_f.plot(vvs, fonsets_ygap, label="constant prephonatory gap")

ax_p.set_xlabel("$v$")
ax_f.set_xlabel("$v$")
ax_p.set_ylabel("$p_\mathrm{on}$ [Pa]")
ax_f.set_ylabel("$f_\mathrm{on}$ [Hz]")
ax_p.legend()
# for ax in axs.flat:
#     ax.legend()

# ax_p.xaxis.set_tick_params(labelbottom=False)

fig.tight_layout()

fig.savefig(f'{FIG_DIR}/OnsetPressureVsSwelling.{FIG_EXT}')

In [ ]:
dx_cover = get_dx(MODEL, 'cover')
def_grad = ufl.grad(MODEL.solid.forms['coeff.state.u1'])
j = ufl.det(def_grad + dfn.Identity(2))

MODEL.solid.set_fin_state(state0)
v0 = dfn.assemble(j*dx_cover)

MODEL.solid.set_fin_state(state_fp)
v1 = dfn.assemble(j*dx_cover)

print(v0, v1, v1/v0)

#### DEBUG - weird behaviour at high psub for finding fixed points

The fixed point solution seems to be unstable at high $p_{sub}$ for unclear reasons.

In [ ]:
psub = 100*10
control = DYN_MODEL.control.copy()
control['psub'] = psub
xfp, info = libhopf.solve_fp(DYN_MODEL, control, prop, xfp_0=statefp_0, psub_incr=1e4, method='newton')
print(info)
xfp.print_summary()

In [ ]:
DYN_MODEL.set_state(xfp)
DYN_MODEL.set_prop(prop)
DYN_MODEL.fluid.control.print_summary()

xref = DYN_MODEL.solid.forms['mesh.mesh'].coordinates()
u = np.array(xfp.sub['u'])[VERT_TO_VDOF].reshape(-1, 2)
xcur = xref+u

print(f"xmax/ymax = ", xcur[:, 0].max(), xcur[:, 1].max())

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.triplot(xcur[:, 0], xcur[:, 1], MESH.cells())

#### Transient solution near onset

You should see that the transient solution begins self-oscillating around the
pressure where the growth rate > 0  in the above plots.

In [ ]:
# TODO: Add a small snippet to check that the transient model agrees with
# the onset prediction
print(model)
control = model.control.copy()
control[:] = 0
control['psub'] = 200*10
# control.print_summary()
# prop.print_summary()
model.dt = 1e-5
model.set_control(control)
model.set_prop(prop)
state_static, info = static.static_coupled_configuration_picard(model, control, prop)

times_vec = np.arange(0, 0.02, 5e-5)
times = blockvec.BlockVector((times_vec,))
print(times.bshape)

In [ ]:
from femvf.postprocess.base import TimeSeries
from femvf.postprocess.solid import MinGlottalWidth

with sf.StateFile(model, 'temp.h5', mode='w') as f:
    forward.integrate(
        model, f, state_static, [control], prop, times, use_tqdm=True
    )

proc_gw = TimeSeries(MinGlottalWidth(model))
with sf.StateFile(model, 'temp.h5', mode='r') as f:
    gw = proc_gw(f)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(times_vec, gw)

### Compare rates for a linearized vicious cycle

In [ ]:
vcovs = np.array([1.0, 1.025])
psubs = np.array([600, 650])*10

In [ ]:
# NOTE: The swelling factor, v, is defined so that if v=1.1, the total volume of
# the swelling part increases by 10%

# dSPL/dv

# dSPL/dpsub

# dphonotrauma/dv

### Plot glottal width signals vs. $v$

In [ ]:
vcovs = np.array([1.0, 1.025, 1.05, 1.075, 1.1])
mcov = -0.8

pdefault = DEFAULT_PARAM.substitute(
    {'vcov': 1.0, 'psub': 400*10, 'dt': 5e-5, 'tf': 0.5}
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(0.75*FIG_LX_WIDE, FIG_LY))

params = [
    {'vcov': vcov, 'mcov': mcov} 
    for vcov, mcov in ittls.product(vcovs, [mcov])
]
ts = load_measures(DATA, 'time.t', params, pdefault)
gws = load_measures(DATA, 'time.gw', params, pdefault)
fs = load_measures(DATA, 'fund_freq', params, pdefault)

for vcov, t, gw, f in zip(vcovs, ts, gws, fs):
    idx = slice(0, None)
    t, gw = t[idx], gw[idx]
    fund_freq = f

    ax.plot(
        t, gw, label=f'$v={vcov}$'
        f', $f_\mathrm{{o}}={fund_freq:.1f} \pm {dfreq:.1e} Hz$'
    )

ax.legend(loc='lower left', bbox_to_anchor=(0, 1))

# ax.set_xlim(0.4, 0.5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Glottal width [cm]")
fig.tight_layout()

fig.savefig(f'{FIG_DIR}/GlottalWidthVsV.{FIG_EXT}', dpi=200)

### Animate kinematics for a particular case

In [ ]:
mcov = 0.0
vcovs = [1.0, 1.3]

fnames = [form_fname(MESH_NAME, ECOV, EBOD, vcov, mcov, PSUB) for vcov in vcovs]
print(model.solid.forms['mesh.mesh'].coordinates().shape)

from scipy.signal import find_peaks
from matplotlib import animation

In [ ]:
def tri_to_linexy(tri):
    x, y, edges = (tri.x, tri.y, tri.edges)
    tri_lines_x = np.insert(x[edges], 2, np.nan, axis=1).ravel()
    tri_lines_y = np.insert(y[edges], 2, np.nan, axis=1).ravel()
    return tri_lines_x, tri_lines_y

def plot_update(f, t, offset, artist, dt):
    ii = int(round(t/dt))
    xcur = XREF + f.get_state(offset+ii).sub['u']
    xcur_vert = xcur[VERT_TO_VDOF].reshape(-1, 2)

    # mutate the triplot lines in artists[0]
    tri_frame = tri.Triangulation(xcur_vert[:, 0], xcur_vert[:, 1], CELLS)
    x, y = tri_to_linexy(tri_frame)
    artist[0].set_xdata(x)
    artist[0].set_ydata(y)
    return artist

In [ ]:
XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY))

gs = mpl.gridspec.GridSpec(2, len(vcovs))
axs_vid = np.array([fig.add_subplot(gs[0, ii]) for ii in range(gs.ncols)])
axs_gw = np.array([fig.add_subplot(gs[1, ii]) for ii in range(gs.ncols)])
axs = np.stack([axs_vid, axs_gw], axis=0)

for ax in axs_vid.flat[1:]:
    axs_vid.flat[0].sharex(ax)
    axs_vid.flat[0].sharey(ax)

for ax in axs_gw[1:]:
    axs_gw.flat[0].sharex(ax)
    axs_gw.flat[0].sharey(ax)

gws = [DATA[f'{fname}/gw'][:] for fname in fnames]
amps = [np.max(gw[-5000:]) for gw in gws]
last_cycle_peaks = [
    find_peaks(gw, height=0.9*amp)[0][-4:]
    for gw, amp in zip(gws, amps)
]

for ax in axs[:, 1:].flat:
    ax.tick_params(labelleft=False)
for ax in axs_vid.flat[1:]:
    ax.tick_params(labelleft=False)

for ax, gw, peaks in zip(axs_gw, gws, last_cycle_peaks):
    ax.plot(gw[peaks[0]:peaks[-1]])

for ax in axs_gw:
    ax.set_xlabel("$t/T$")
axs_gw[0].set_ylabel("Glottal width [cm]")
axs_vid[0].set_ylabel("y [cm]")

artists = [ax.triplot(trian, lw=0.75) for ax in axs_vid]
for ax in axs_vid:
    ax.set_xlabel("x [cm]")
    ax.set_aspect(1)
    ax.set_xlim(-0.1, 0.9)
    ax.set_ylim(0.0, 0.7)

fig.tight_layout()


dts = [3/(peaks[-1]-peaks[0]) for peaks in last_cycle_peaks]
offsets = [peaks[0] for peaks in last_cycle_peaks]



NFRAME = 10
with (
        sf.StateFile(model, f'{OUT_DIR}/{fnames[0]}.h5', mode='r') as f1,
        sf.StateFile(model, f'{OUT_DIR}/{fnames[1]}.h5', mode='r') as f2
    ):
    fs = [f1, f2]

    ii = 1
    plot_update(fs[ii], 0.0, offsets[ii], artists[ii], dts[ii])

    def _update(frame):
        _artists = []
        t = 3*frame/NFRAME
        for f, artist, offset, dt in zip(fs, artists, offsets, dts):
            _artist = plot_update(f, t, offset, artist, dt)
            _artists.append(_artist)

        return np.concatenate(_artists).tolist()

    _update(4)

    ani = animation.FuncAnimation(
        fig,
        _update,
        blit=True,
        frames=np.arange(NFRAME)
    )
    ani.save(f'{FIG_DIR}/v{vcov:.2e}_m{mcov:.2e}.mp4', dpi=300, fps=60)

# for f in fs:
#     f.close()

In [ ]:
## Animate motion
fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL)
ax.set_xlim(-0.25, 1.0)
ax.set_ylim(0, 0.6)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
ax.set_aspect(1.0, adjustable='box')
ax.axhline(0.55, color='k', ls='-.')
fig.tight_layout()

XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)
artists = ax.triplot(trian, lw=0.75)

vcov = 1.3
fname = form_fname(MESH_NAME, ECOV, EBOD, vcov, mcov, PSUB)
with sf.StateFile(model, f'{OUT_DIR}/{fname}.h5', mode='r') as f:
    def _plot_update(frame):
#         xcur = XREF + f.get_state(frame)['u']
#         xcur_vert = xcur[VERT_TO_VDOF].reshape(-1, 2)

#         # mutate the triplot lines in artists[0]
#         tri_frame = tri.Triangulation(xcur_vert[:, 0], xcur_vert[:, 1], CELLS)
#         x, y = tri_to_linexy(tri_frame)
#         artists[0].set_xdata(x)
#         artists[0].set_ydata(y)
        dt = 1
        offset = 0
        return plot_update(f, frame, offset, artists, dt)
        # return artists
    # plot_update(1000)

    ani = animation.FuncAnimation(
        fig,
        _plot_update,
        blit=True,
        frames=f.size + np.arange(-1000, 0)
    )
    ani.save(f'{FIG_DIR}/v{vcov:.2e}_m{mcov:.2e}.mp4', dpi=450, fps=60)

### Plot swollen geometries vs. $v$

In [ ]:
vcovs = np.array([1.0, 1.025, 1.05, 1.075, 1.1])
mcov = -0.8

pdefault = DEFAULT_PARAM

In [ ]:
# Load the first state/displacement for each swelling case
# ; this is the swelling displacement because we solved for the
# static equilibrium before integrating in time
mcov = -0.8
params = [
    {'vcov': vcov, 'mcov': mcov} for vcov, mcov in ittls.product(vcovs, mcovs)
]

swelling_displacements = {}
for param in params:
    with sf.StateFile(model, f'{OUT_DIR}/{param.to_str()}.h5') as f:
        swelling_displacements[param['vcov']] = f.get_state(0).sub['u']

        # NOTE: for debugging
        # prop = f.get_prop()
        # prop.print_summary()

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL, sharex=True)
elif FIG_TYPE == 'presentation':
    fig, ax = plt.subplots(1, 1, figsize=(FIG_LX_WIDE/2, FIG_LY), sharex=True)
# fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL)

scale = 2
vcovs = np.array([1.0, 1.3])
for vcov, color, marker, ls in zip(vcovs, COLOR_CYCLE, MARKER_CYCLE, LS_CYCLE):
    u_dof = swelling_displacements[vcov]
    xcur_dof = XREF + scale*u_dof
    xcur_ver = xcur_dof[VERT_TO_VDOF].reshape(-1, 2)

    # This plots the whole triangulation/mesh
    # triang = tri.Triangulation(xcur_ver[:, 0], xcur_ver[:, 1], triangles=CELLS)
    # ax.triplot(triang, color=color)

    # Plot the medial and interface surfaces
    vert_med = np.concatenate([VERT_MED, VERT_MED[:1]])
    kwargs_line_style = {
        'color': color,
        'ls': ls
        # 'marker': marker,
        # 'markevery': 5
    }
    ax.plot(*xcur_ver[vert_med].T, **kwargs_line_style)
    ax.plot(*xcur_ver[VERT_INT].T, **kwargs_line_style)

# Create a custom legend for each swelling level
from matplotlib.lines import Line2D
lines = [
    Line2D([0], [0], color=color, ls=ls)
    for color, ls in zip(COLOR_CYCLE[:len(vcovs)], LS_CYCLE)
]
labels = [f"$v$={vcov:.2f}" for vcov in vcovs]
ax.legend(lines, labels)

ax.set_aspect(1)
ax.set_xlabel("x $[\mathrm{cm}]$")
ax.set_ylabel("y $[\mathrm{cm}]$")
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/SwollenGeometryVsV.{FIG_EXT}", dpi=DPI)

In [ ]:
## Plot for P50 presentation
vs = [1., 1.3]

for v in vs:
    u_dof = swelling_displacements[v]
    xcur_dof = XREF + scale*u_dof
    xcur_ver = xcur_dof[VERT_TO_VDOF].reshape(-1, 2)

    fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL)
    ax.triplot(*xcur_ver.T, CELLS, lw=0.75)
    ax.triplot(*xcur_ver.T, CELLS[5:6], lw=1, color='r')
    ax.set_aspect(1)
    ax.yaxis.set_tick_params(label1On=False, tick1On=False)
    ax.xaxis.set_tick_params(label1On=False, tick1On=False)

    fig.savefig(f'{FIG_DIR}/SwellingV{v:.1f}.{FIG_EXT}')

### Plot mass and volume vs. $v$

In [ ]:
from matplotlib.lines import Line2D

vcovs = np.array([1.0, 1.025, 1.05, 1.075, 1.1])
mcov = -0.8

pdefault = DEFAULT_PARAM.substitute({'psub': 400*10})

In [ ]:
params = [{'vcov': vcov, 'mcov': mcov, 'tf': 0.5} for vcov in vcovs]
masses_cov = load_measures_np(DATA, 'mass_cover', params, pdefault)
masses_tot = load_measures_np(DATA, 'mass_total', params, pdefault)

vols_cov = load_measures_np(DATA, 'vol_cover', params, pdefault)
vols_tot = load_measures_np(DATA, 'vol_total', params, pdefault)

vols_cov = load_measures_np(DATA, 'vol_cover', params, pdefault)

In [ ]:
masses_cov
vols_cov

In [ ]:
## Plot showing cover volume vs swelling
lss = ['-',  '-.', ':']
fig, ax = plt.subplots(1, 1, sharex=True, figsize=(FIG_LX, FIG_LY))
ax.plot(vcovs, vols_cov, ls=lss[0], label="absolute cover volume", color=COLOR_CYCLE[0])
ax_twin = ax.twinx()
ax_twin.plot(vcovs, 100*vols_cov/vols_tot, ls=lss[1], color=COLOR_CYCLE[1], label="relative to total VF volume")
ax_twin.plot(vcovs, 100*(vols_cov-vols_cov[0])/vols_cov[0], ls=lss[2], color=COLOR_CYCLE[1], label="relative to initial cover volume")
ax_twin.set_ylabel("Cover volume $[\si{\percent}]$")
ax_twin.set_ylim(0, 30)

# ax_twin.legend(loc=(0, 1.05))
# ax.legend(loc=())

colors = [COLOR_CYCLE[0]] + [COLOR_CYCLE[1]]*2

lines = [Line2D([], [], color=color, ls=ls) for color, ls in zip(colors, lss)]
labels = [
    'absolute (left axis) volume $[\si{\cm^2}]$ ',
    'relative (right axis) \n to total VF volume $[\si{\percent}]$ ',
    'relative (right axis) \n to initial cover volume $[\si{\percent}] $'
]
# ax.legend(lines, labels, loc=(0, 1.05), handlelength=3)

ax.legend(lines, labels, loc='lower left', bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')

ax.set_ylabel("Cover volume $[\si{\cm^2}]$")
ax.set_xlabel("$v$")

fig.tight_layout()
fig.savefig(f"{FIG_DIR}/CoverVolumeVsV.{FIG_EXT}")

In [ ]:
## Plot showing cover volume + mass vs swelling
fig, axs = plt.subplots(2, 1, sharex=True, figsize=SIZE_MED)
axs[0].plot(VCOVS, masses_cov)
ax_twin = axs[0].twinx()
ax_twin.plot(VCOVS, 100*masses_cov/masses_tot, ls=':')
ax_twin.set_ylabel("% total")

axs[1].plot(VCOVS, vols_cov)
ax_twin = axs[1].twinx()
ax_twin.plot(VCOVS, 100*vols_cov/vols_tot, ls=':')
ax_twin.set_ylabel("% total")

axs[0].set_ylabel("Cover mass [\si{\g}]")
axs[1].set_ylabel("Cover volume [\si{\cm^2}]")

axs[1].set_xlabel("$v$ [1]")

fig.tight_layout()
# fig.savefig(f"{FIG_DIR}/CoverMassVsV.png")
# fig.savefig(f"{FIG_DIR}/CoverMassVsV.pdf")

### Plot (SPL, $f_\mathrm{o}$, Amplitude) vs. $v$

In [ ]:
vcovs = np.array([1.0, 1.025, 1.05, 1.075, 1.1])
mcovs = np.array([0.0, -0.8])
default_param = DEFAULT_PARAM.substitute({'psub': 400*10, 'ModifyEffect': '', 'tf': 0.5})

In [ ]:
param_dicts = [{'vcov': vcov} for vcov in VCOVS]
prms_trends = {
    mcov: load_measures_np(
        DATA, 'prms', param_dicts, default_param.substitute({'mcov': mcov})
    )
    for mcov in mcovs
}
data_f0 = {
    mcov: load_measures_np(
        DATA, 'fund_freq', param_dicts, default_param.substitute({'mcov': mcov})
    )
    for mcov in mcovs
}
data_df0 = {
    mcov: load_measures_np(
        DATA, 'dfreq', param_dicts, default_param.substitute({'mcov': mcov})
    )
    for mcov in mcovs
}
data_amp = {
    mcov: load_measures_np(
        DATA, 'amplitude', param_dicts, default_param.substitute({'mcov': mcov})
    )
    for mcov in mcovs
}

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(3, 1, figsize=SIZE_MED, sharex=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 2, figsize=(FIG_LX_WIDE, FIG_LY), sharex=True)

for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
    y = data_f0[mcov]
    dy = data_df0[mcov]
    kwargs_line = {
        'ls': ls,
        'color': color
    }

    axs[0].plot(VCOVS, y, label=f"$\\bar{{m}}'={mcov:.1f}$", **kwargs_line)
    print(y)

    prmss = prms_trends[mcov]
    spls = 20*np.log10(prmss/20e-6)
    axs[1].plot(vcovs, spls, label=f"$\\bar{{m}}'={mcov:.1f}$", **kwargs_line)

    if FIG_TYPE != 'presentation':
        amps = data_amp[mcov]
        axs[2].plot(vcovs, amps, label=f"$\\bar{{m}}'={mcov:.1f}$", **kwargs_line)

axs[0].legend(loc='lower left', ncol=3, bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')
axs[0].set_ylabel("$f_o$ $[\si{\Hz}]$")
axs[1].set_ylabel("$\mathrm{SPL}$ $[\si{\dB}]$")

if FIG_TYPE != 'presentation':
    axs[2].set_ylabel("Amplitude $[\si{\cm}]$")

n = -1
mcov = 0.0
v = vcovs[n]
f0s = [data_f0[mcov][n] for mcov in mcovs[[0, -1]]]
spls = [20*np.log10(prms_trends[mcov]/20e-6)[n] for mcov in mcovs[[0, -1]]]
annotate_vertical_trend(axs[0], "increasing softening", v, *f0s)
annotate_vertical_trend(axs[1], "increasing softening", v, *spls)


if FIG_TYPE == 'manuscript':
    axs[-1].set_xlabel("$v$")
elif FIG_TYPE == 'presentation':
    for ax in axs:
        ax.set_xlabel("$v$")
        ax.set_xticks(vcovs)

fig.tight_layout()
fig.savefig(f"{FIG_DIR}/F0VsV.{FIG_EXT}")

### Summarize $f_\mathrm{o}$ and phonotrauma measure

In [ ]:
vcovs = np.array([1.0, 1.025, 1.05, 1.075, 1.1])
mcovs = np.array([0.0, -0.8])
default_param = DEFAULT_PARAM.substitute({'psub': 400*10, 'ModifyEffect': '', 'tf': 0.5})

In [ ]:

param_dicts = [{'vcov': vcov} for vcov in vcovs]
dat_f0 = {
    mcov: load_measures_np(
        DATA, 'fund_freq', param_dicts,
        default_param.substitute({'mcov': mcov})
    )
    for mcov in mcovs
}

dat_spl = {
    mcov: load_measures_np(
        DATA, 'spl', param_dicts,
        default_param.substitute({'mcov': mcov})
    )
    for mcov in mcovs
}

dat_wvisc = {
    mcov: load_measures_np(
        DATA, 'time.spatial_stats_viscous', param_dicts,
        default_param.substitute({'mcov': mcov})
    )
    for mcov in mcovs
}

In [ ]:
fig = plt.figure(
    figsize=(FIG_LX_WIDE/2, FIG_LY/2), constrained_layout=True
)

gs = mpl.gridspec.GridSpec(1, 2, figure=fig)
ax_acous = fig.add_subplot(gs[0])
ax_wvisc = fig.add_subplot(gs[1])
ax_acous.xaxis.set_tick_params(labelbottom=False)

for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
    kwargs = {
        'color': color, 'ls': ls,
        'label': f"$\\bar{{m}}'={mcov:.1f}$"
    }
    ax_acous.plot(VCOVS, dat_f0[mcov], **kwargs)

    N = dat_wvisc[mcov].shape[-1]
    ys = np.mean(dat_wvisc[mcov]['avg'][..., -N//2:], axis=-1)
    ax_wvisc.plot(VCOVS, ys*1e-7/(1e-2**2), **kwargs)

    ax_acous.legend()

n = 3
v = vcovs[n]
wviscs = [
    1e-7/(1e-2**2)*np.mean(dat_wvisc[mcov]['avg'][..., -N//2:][n], axis=-1)
    for mcov in mcovs[[0, -1]]
]
f0s = [dat_f0[mcov][n] for mcov in mcovs[[0, -1]]]
annotate_vertical_trend(ax_acous, "increasing softening", v, *f0s)
annotate_vertical_trend(ax_wvisc, "increasing softening", v, *wviscs)

ax_acous.set_ylabel("$f_o$ [Hz]")
ax_wvisc.set_ylabel(r"$\hat{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")

ax_wvisc.set_xlabel("$v$")

fig.savefig(f'{FIG_DIR}/FundamentalFrequencyandPhonotraumaSummary.{FIG_EXT}')

In [ ]:
fig = plt.figure(
    figsize=(FIG_LX_WIDE/2, FIG_LY/2), constrained_layout=True
)

gs = mpl.gridspec.GridSpec(2, 1, figure=fig)
ax_acous = fig.add_subplot(gs[0])
ax_wvisc = fig.add_subplot(gs[1])
ax_acous.xaxis.set_tick_params(labelbottom=False)

for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
    kwargs = {
        'color': color, 'ls': ls,
        'label': f"$\\bar{{m}}'={mcov:.1f}$"
    }
    ax_acous.plot(VCOVS, dat_spl[mcov], **kwargs)

    N = dat_wvisc[mcov].shape[-1]
    ys = np.mean(dat_wvisc[mcov]['avg'][..., -N//2:], axis=-1)
    ax_wvisc.plot(VCOVS, ys*1e-7/(1e-2**2), **kwargs)

    ax_acous.legend()

n = 3
v = vcovs[n]
wviscs = [
    1e-7/(1e-2**2)*np.mean(dat_wvisc[mcov]['avg'][..., -N//2:][n], axis=-1)
    for mcov in mcovs[[0, -1]]
]
f0s = [dat_spl[mcov][n] for mcov in mcovs[[0, -1]]]
annotate_vertical_trend(ax_acous, "increasing softening", v, *f0s)
annotate_vertical_trend(ax_wvisc, "increasing softening", v, *wviscs)

ax_acous.set_ylabel("$\mathrm{SPL}$ [dB]")
ax_wvisc.set_ylabel(r"$\hat{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")
ax_wvisc.set_xlabel("$v$")

fig.savefig(f'{FIG_DIR}/SPLandPhonotraumaSummary.{FIG_EXT}')

### Dynamics over representative cycle

Plot dynamics over a cycle (eg. contact stress over one cycle, maybe average spatial stress field over one cycle)



In [ ]:
measure_names = [
    'time', 'gw', 'period_sample',
    'signal_spatial_stats_con_p', 'signal_spatial_stats_con_a',
    'signal_spatial_stats_viscous', 'signal_savg_viscous_rate',
    'signal_spatial_stats_vm'
]
dat = {
    f'{vcov:.2f}_{meas_name}': load_measures(
        DATA, f'{meas_name}', {'vcov': vcov, 'mcov': mcov}
    )
    for meas_name in measure_names for vcov in VCOVS
}

# `contact_stats` contains (max pc, avg pc, total, pc, areac) along the last axis

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=SIZE_MED, sharex=True)

for v in VCOVS[::2]:
    _period = dat[f'{v:.2f}_period_sample']
    # print(_period)
    PERIOD = int(round(_period))
    N_PERIODS = 5

    tt = np.linspace(0, N_PERIODS, PERIOD*N_PERIODS)
    yy = dat[f'{v:.2f}_signal_spatial_stats_con_a']['total'][-N_PERIODS*PERIOD:]/10/1e3
    axs[0].plot(tt, yy, label=f"$v$={v:.2f}")

    yy = dat[f'{v:.2f}_signal_spatial_stats_vm']['avg'][-N_PERIODS*PERIOD:]/10/1e3
    axs[1].plot(tt, yy, label=f"$v$={v:.2f}")

ylabels = [
    "$\overline{p_\mathrm{c}}$ [kPa]",
    "$\overline{\sigma_\mathrm{vm}}$ [kPa]"
]
for ax, ylabel in zip(axs, ylabels):
    ax.set_ylabel(ylabel)
axs[-1].set_xlabel("Time [period]")

axs[0].legend()

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/ContactVMCycleDynamicsVsV.{FIG_EXT}')

### Plot fields

In [ ]:
model = MODEL

#### von Mises stress

In [ ]:
# Specify the case(s) of interest

## TODO: Try to ameliorate numerical stress artifacts (likely due to near-incompressibilty)
# mcov = 0.0
meas_names = [
    'field.tavg_vm',
    'field.tavg_hydrostatic',
    'field.tini_vm',
    'field.tini_hydrostatic',
    # 'field_fp_vm',
    # 'field_fp_hydrostatic'
]
if FIG_TYPE == 'manuscript':
    vcovs = [1.0, 1.15, 1.3]
    mcovs = MCOVS[::2]

    vcovs = [1.0, 1.1, 1.2, 1.3]
    mcovs = [0, -0.8, -1.6]
elif FIG_TYPE == 'presentation':
    vcovs = [1.0, 1.1, 1.15, 1.3]
    mcovs = [-0.8]

dat = {
    f'{mcov:.2e}/{meas_name}': load_measures(
        DATA, f'{meas_name}', [
            [{'vcov': vcov, 'mcov': mcov} for vcov in vcovs]
        ]
    )
    for meas_name in meas_names
    for mcov in mcovs
}

In [ ]:
field_prefix = 'ini'
field_prefix = 'avg'
# field_prefix = 'fp'

figsize = (FIG_LX_WIDE, 13*CM)
# figsize = (8, 4)
# fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, figsize=figsize)

ncol = len(vcovs)
nrow = len(mcovs)
gs = mpl.gridspec.GridSpec(
    nrow+2, ncol+1,
    width_ratios=[1]*ncol + [0.05],
    height_ratios=[0.05]*2+[1]*nrow
)
fig = plt.figure(figsize=figsize)

XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)

axs = np.array(
    [
        [fig.add_subplot(gs[ii+2, jj]) for jj in range(ncol)]
        for ii in range(nrow)
    ],
    dtype=object
)

for ax in axs.flat:
    ax.sharex(axs.flat[0])
    ax.sharey(axs.flat[0])

ax_cbar_abs = fig.add_subplot(gs[0, 0])
ax_cbar_diff = fig.add_subplot(gs[0, 1])

ys = np.array([
    dat[f'{mcov:.2e}/field_tavg_vm'][ii]/10/1e3
    for mcov in mcovs
    for ii in range(len(vcovs))
]).reshape(len(mcovs), len(vcovs), -1)
ys_diff = ys[:, 1:] - ys[:, 0:1]
ys_diff_percent = (ys[:, 1:] - ys[:, 0:1])/ys[:, 0:1]
artists_diff = plot_tripcolor_seq(axs[:, 1:], trian, ys_diff, zmax=0.3)

ys_abs = ys[:, 0]
artists_abs = plot_tripcolor_seq(axs[:, 0], trian, ys_abs, zmax=None)

# for ax, vcov in zip(_axs[1:], vcovs[1:]):
#     ax.text(0.05, 0.95, f"$v={vcov:.2f}$\n$m'={mcov:.2f}$", transform=ax.transAxes, ha='left', va='top')

# ys_abs = np.array([y-ys[0] for y in ys[1:]])
# artists_diff = plot_tripcolor_seq(_axs[1:], trian, ys_diff)

cbar = fig.colorbar(artists_diff, cax=ax_cbar_diff, orientation='horizontal')
ax_cbar_diff.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
ax_cbar_diff.set_xlabel(r"$\Delta \tilde{\sigma}_\mathrm{vm}$ [\si{\kPa}]")

cbar = fig.colorbar(artists_abs, cax=ax_cbar_abs, orientation='horizontal')
ax_cbar_abs.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
ax_cbar_abs.set_xlabel(r"$\tilde{\sigma}_\mathrm{vm}$ [\si{\kPa}]")

# Add annotations for changing v + m
ax = fig.add_subplot(gs[1, :-1])
ax.axis('off')
ax.annotate(
    "increasing swelling", ha='right', va='center',
    xytext=(0.4, 1.3), xy=(.6, 1.3), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'}
)

ax = fig.add_subplot(gs[2:, -1])
ax.axis('off')
ax.annotate(
    "increasing softening", ha='center', va='bottom', rotation=-90,
    xytext=(1.15, 0.6), xy=(1.15, 0.4), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'},
)

for ax in axs.flat:
    ax.set_aspect(1, adjustable='box')

for ax in axs[:, 1:].flat:
    ax.tick_params(axis='y', labelleft=False)

for ax in axs[:-1, :].flat:
    ax.tick_params(axis='x', labelbottom=False)

for ax, mcov in zip(axs[:, 0], mcovs):
    ax.set_ylabel(r"y [\si{\cm}]")
for ax, vcov in zip(axs[-1, :], vcovs):
    ax.set_xlabel(r"x [\si{\cm}]")


for ax, mcov in zip(axs[:, -1], mcovs):
    text_kwargs = {
        'transform': ax.transAxes,
        'ha': 'left',
        'va': 'center'
    }
    ax.text(1, 0.5, f"$\\bar{{m}}'={mcov:.2f}$", rotation=-90, **text_kwargs)
for ax, vcov in zip(axs[0, :], vcovs):
    text_kwargs = {
        'transform': ax.transAxes,
        'ha': 'center',
        'va': 'bottom'
    }
    ax.text(0.5, 1.0, f"\n$v$={vcov:.2f}", **text_kwargs)

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/StressFieldVsV.{FIG_EXT}')

#### Viscous dissipation

In [ ]:
if FIG_TYPE == 'manuscript':
    vcovs = [1.0]
    mcovs = [0.0]

    # vcovs = [1.0, 1.1, 1.2, 1.3]
    # mcovs = [0.0, -0.8, -1.6]
elif FIG_TYPE == 'presentation':
    vcovs = [1.0, 1.1, 1.2, 1.3]
    mcovs = [-0.8, MCOVS[-1]]

meas_name = 'field.tavg_viscous_rate'
dat = {
    f'{mcov:.2e}/{meas_name}': load_measures(
        DATA, f'{meas_name}', [{'vcov': vcov, 'mcov': mcov} for vcov in vcovs]
    )
    for mcov in mcovs
}

In [ ]:
figsize = (FIG_LX_WIDE, FIG_LY)
ncol = len(vcovs)
nrow = len(mcovs)
gs = mpl.gridspec.GridSpec(
    nrow+2, ncol+1,
    width_ratios=[1]*ncol + [0.05],
    height_ratios=[0.05]*2+[1]*nrow
)
fig = plt.figure(figsize=figsize)

XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)

axs = np.array(
    [
        [fig.add_subplot(gs[ii+2, jj]) for jj in range(ncol)]
        for ii in range(nrow)
    ],
    dtype=object
)
ax_cbar_abs = fig.add_subplot(gs[0, 0])
ax_cbar_diff = fig.add_subplot(gs[0, 1])

for ax in axs.flat:
    ax.sharex(axs.flat[0])
    ax.sharey(axs.flat[0])

ys = np.array([
    dat[f'{mcov:.2e}/field.tavg_viscous_rate'][ii]*1e-7/(1e-2 ** 2)
    for mcov in mcovs
    for ii in range(len(vcovs))
]).reshape(len(mcovs), len(vcovs), -1)
ys_diff = ys[:, 1:] - ys[:, 0:1]
ys_diff_percent = (ys[:, 1:] - ys[:, 0:1])/ys[:, 0:1]
artists_diff = plot_tripcolor_seq(axs[:, 1:], trian, ys_diff, zmax=50)

ys_abs = ys[:, 0]
artists_abs = plot_tripcolor_seq(axs[:, 0], trian, ys_abs)

cbar = fig.colorbar(artists_diff, cax=ax_cbar_diff, orientation='horizontal')
ax_cbar_diff.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
ax_cbar_diff.set_xlabel(r"$\Delta \tilde{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")

cbar = fig.colorbar(artists_abs, cax=ax_cbar_abs, orientation='horizontal')
ax_cbar_abs.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
ax_cbar_abs.set_xlabel(r"$\tilde{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")

# Add annotations for changing v + m
ax = fig.add_subplot(gs[1, :-1])
ax.axis('off')
ax.annotate(
    "increasing swelling", ha='right', va='center',
    xytext=(0.4, 1.3), xy=(.6, 1.3), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'}
)

ax = fig.add_subplot(gs[2:, -1])
ax.axis('off')
ax.annotate(
    "increasing softening", ha='center', va='bottom', rotation=-90,
    xytext=(1.15, 0.6), xy=(1.15, 0.4), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'},
)

for ax in axs.flat:
    ax.set_aspect(1, adjustable='box')

for ax in axs[:, 1:].flat:
    ax.tick_params(axis='y', labelleft=False)

for ax in axs[:-1, :].flat:
    ax.tick_params(axis='x', labelbottom=False)

for ax, mcov in zip(axs[:, 0], mcovs):
    ax.set_ylabel(r"$y$ $[\si{\cm}]$")
for ax, vcov in zip(axs[-1, :], vcovs):
    ax.set_xlabel(r"$x$ $[\si{\cm}]$")

for ax, mcov in zip(axs[:, -1], mcovs):
    text_kwargs = {
        'transform': ax.transAxes,
        'ha': 'left',
        'va': 'center'
    }
    ax.text(1, 0.5, f"$\\bar{{m}}'={mcov:.2f}$", rotation=-90, **text_kwargs)
for ax, vcov in zip(axs[0, :], vcovs):
    text_kwargs = {
        'transform': ax.transAxes,
        'ha': 'center',
        'va': 'bottom'
    }
    ax.text(0.5, 1.0, f"\n$v$={vcov:.2f}", **text_kwargs)

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/ViscousRateVsV.{FIG_EXT}')

In [ ]:
import paraview

#### Viscous dissipation distribution

In [ ]:
if FIG_TYPE == 'manuscript':
    vcovs = [1.0, 1.15, 1.3]
    mcovs = MCOVS[::2]

    vcovs = [1.0, 1.1, 1.2, 1.3]
    mcovs = [0.0, -0.8, -1.6]
elif FIG_TYPE == 'presentation':
    vcovs = [1.3]
    mcovs = [-0.8]

meas_name = 'field_tavg_viscous_rate'
dat = load_measures(
    DATA, f'{meas_name}', [{'vcov': vcov, 'mcov': mcov} for vcov, mcov in ittls.product(vcovs, mcovs)]
)

In [ ]:
XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)

gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[0.02, 1])
fig = plt.figure(figsize=(14.7*CM, 13*CM))

ax = fig.add_subplot(gs[1, 0])
artist = ax.tripcolor(trian, dat*1e-7/(1e-2 ** 2))
ax.set_aspect(1)

ax.set_xlabel("$x \, [\mathrm{cm}]$")
ax.set_ylabel("$y \, [\mathrm{cm}]$")

ax_cbar = fig.add_subplot(gs[0, 0])
fig.colorbar(artist, cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel(r"$\tilde{w}_\mathrm{visc}$ $[\mathrm{W}\mathrm{m}^{-2}]$")

fig.savefig(f"{FIG_DIR}/ViscousDissipationExample.{FIG_EXT}")

#### Contact pressure

In [ ]:
## Compute the 's' coordinate for the DG0 contact pressure
forms = MODEL.solid.forms
mesh = forms['mesh.mesh']
facet_function = forms['mesh.facet_function']
facet_data = forms['mesh.facet_label_to_id']
dofmap = forms['fspace.scalar_dg0'].dofmap()

# Find facets along the traction surface and associated DG0 dofs
facet_coords = []
facet_cell_neighbor_idxs = []
for facet in dfn.facets(mesh):
    facet_idx = facet.index()
    if facet_function[facet_idx] == facet_data['pressure']:
        facet_coord = facet.midpoint().array()[:2]
        facet_coords.append(facet_coord)

        cell_neighbor_idx = facet.entities(2)
        assert len(cell_neighbor_idx) == 1
        cell_neighbor_idx = cell_neighbor_idx[0]
        facet_cell_neighbor_idxs.append(cell_neighbor_idx)

facet_coords = np.array(facet_coords)
facet_cell_neighbor_idxs = np.array(facet_cell_neighbor_idxs)
facet_cell_neighbor_dofs = np.array(dofmap.entity_dofs(mesh, 2, facet_cell_neighbor_idxs))

# Sort the DG0 dofs in streamwise increasing order
from femvf.meshutils import sort_vertices_by_nearest_neighbours
idx_sort = sort_vertices_by_nearest_neighbours(facet_coords, np.array([0,0]))
facet_coords = facet_coords[idx_sort]
facet_cell_neighbor_dofs = facet_cell_neighbor_dofs[idx_sort]
_facet_coords = np.concatenate([facet_coords, np.array([[0, 0]])], axis=0)
ds = np.linalg.norm(_facet_coords[1:]-_facet_coords[:-1], axis=-1)
facet_s = np.cumsum(ds)

print(facet_s.shape)
print(facet_cell_neighbor_dofs.shape)

In [ ]:
# mcov = 0.0
mcovs = [0, -0.8, -1.6]
vcovs = [1.0, 1.05, 1.1]

mcovs = [0, -0.8, -1.6]
vcovs = [1.0, 1.1, 1.2, 1.3]

# Load the contact force/area
param_dicts = [{'vcov': vcov} for vcov in vcovs]
_dat_pcfield = {
    mcov: load_measures_np(
        DATA, 'field_tavg_pc', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in mcovs
}
_dat_pc = {
    mcov: load_measures_np(
        DATA, 'signal_spatial_stats_con_p', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in mcovs
}
_dat_pa = {
    mcov: load_measures_np(
        DATA, 'signal_spatial_stats_con_a', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in mcovs
}
dat_cq = {
    key: np.sum(y['total'] > 0, axis=-1)/y.shape[-1]
    for key, y in _dat_pa.items()
}

# print(_dat_pc[-0.8].shape)
# print(_dat_pc[-0.8].dtype)
# print(_dat_pc[-0.8]['total'].shape)

# Load the contact pressure field and account for averaging over contact times only
# Convert the unit to kpa
# If multiplying by `1/cq`, the contact pressure is averaged over contact only
dat = {
    mcov: _dat_pcfield[mcov]/10/1e3 * 1/dat_cq[mcov][:, None]
    for mcov in mcovs
}

In [ ]:
## PLot contact surface pressure profiles
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, 8*CM), sharex=True, sharey=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, FIG_LY), sharex=True, sharey=True)

s = model.fluid.s
for ax, mcov in zip(axs, mcovs):
    print(mcov)
    for vcov, pc, cq, marker in zip(vcovs, dat[mcov], dat_cq[mcov], MARKER_CYCLE):
        kwargs_line = {'marker': marker, 'markevery': 5, 'ms': 5}
        pc_medial = pc[facet_cell_neighbor_dofs]
        ax.plot(facet_s, pc_medial, label=f"$v$={vcov:.2f}", **kwargs_line)

axs[0].set_ylabel(r"$\tilde{p}_\mathrm{c}$ [\si{\kPa}]")
axs[0].legend(loc=(0, 1.05), ncol=1)

# Add annotations for changing v + m
gs = mpl.gridspec.GridSpec(1, 3, figure=fig)
ax = fig.add_subplot(gs[0, :])
ax.axis('off')
ax.annotate(
    "increasing softening", ha='right', va='center',
    xytext=(0.4, 1.3), xy=(.6, 1.3), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'}
)

for ax, m in zip(axs, mcovs):
    ax.set_xlim(0.6, 1.0)
    ax.set_xlabel("$s$ [\si{\cm}]")
    ax.text(0.05, 0.95, f"$\\bar{{m}}'={m:.2f}$", transform=ax.transAxes, ha='left', va='top')

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/ContactPressureFieldVsV.{FIG_EXT}')

In [ ]:
## PLot contact surface pressure profiles
if FIG_TYPE == 'presentation':
    fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY))
elif FIG_TYPE == 'manuscript':
    fig = plt.figure(figsize=(FIG_LX_WIDE, 0.3*FIG_LY))
gs = mpl.gridspec.GridSpec(2, 3, height_ratios=[0.1, 1], hspace=0.05)

ax_legend = fig.add_subplot(gs[0, :])
axs = np.array([fig.add_subplot(gs[1, ii]) for ii in range(gs.ncols)])

for ax in axs:
    ax.sharex(axs[0])
    ax.sharey(axs[0])

ax_legend.set_axis_off()
for ax in axs[1:]:
    ax.tick_params(labelleft=False)

# fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, 6*CM), sharex=True, sharey=True)

s = model.fluid.s
xy_med = facet_coords
nn_med = normal(xy_med)
for ax in axs:
    ax.plot(xy_med[:, 0], xy_med[:, 1], color='k')
    ax.plot(xy_med[[0, -1], 0], [0, 0], color='k')
    ax.set_aspect(1)

for ax, mcov in zip(axs, mcovs):
    for vcov, pc in zip(vcovs, dat[mcov]):
        pc_medial = pc[facet_cell_neighbor_dofs]
        scale = -0.2

        pc_n = offset_normal(xy_med, scale*pc_medial[:, None]*nn_med)
        ax.plot(pc_n[:, 0], pc_n[:, 1], label=f"$v$={vcov:.2f}")

        # average contact pressure field
        # ax.plot(s, pc_medial, label=f"$v$={vcov:.1f}")

N = 25
for ax, mcov in zip(axs, mcovs):
    dxy = np.max(dat[mcov])*scale*nn_med[N]
    ax.arrow(*xy_med[N], *dxy, head_width=0)
    pad = np.array([-0.01, 0])
    ax.text(*(xy_med[N] + dxy/4 + pad), f"{np.max(dat[mcov]):.2f} [kPa]", ha='right')
    ax.text(0.05, 0.95, f"$m'={mcov:.2f}$", transform=ax.transAxes, va='top', ha='left')

axs[0].set_ylabel("$y$ [cm]")


labels = [f"$v={v:.2f}$" for v in vcovs]
lines = [mpl.lines.Line2D([0], [0], color=color) for color in COLOR_CYCLE]
ax_legend.legend(lines, labels, loc='lower left', ncol=len(lines))
# axs[0].legend(loc=(0, 1.05), ncol=1, ax=ax_legend)
# axs[0].legend()

for ax in axs:
    ax.set_xlim(0.2, 0.8)
    ax.set_ylim(0.2, 0.7)
    ax.set_xlabel("x [cm]")

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/ContactPressureFieldVsV-Fancy.{FIG_EXT}')

### Contact pressure stats vs $v$

In [ ]:
# Process the trend of contact pressure vs v for each mcov case
default_param = DEFAULT_PARAM.copy()
mcovs = np.array([0, -0.4, -0.8, -1.2, -1.6])
mcovs = np.array([0])
vcovs = VCOVS[:]

param_dicts = [{'vcov': vcov} for vcov in vcovs]
stats_contact_p = {
    mcov: load_measures_np(
        DATA, 'time.spatial_stats_con_p', param_dicts,
        {**default_param, 'mcov': mcov}
    )
    for mcov in mcovs
}
stats_contact_a = {
    mcov: load_measures_np(
        DATA, 'time.spatial_stats_con_a', param_dicts,
        {**default_param, 'mcov': mcov}
    )
    for mcov in mcovs
}
ts = {
    mcov: load_measures_np(
        DATA, 'time.t', param_dicts,
        {**default_param, 'mcov': mcov}
    )
    for mcov in mcovs
}
# fund_freqs = {
#     mcov: load_measures_np(DATA, 'fund_freq', {'vcov': vcovs, 'mcov': mcov})
#     for mcov in mcovs
# }

In [ ]:
def avg_contact_pressure(t, cp_stats, ca_stats):

    # Assume the final axis varies time
    N = cp_stats.shape[-1]//2

    # Load total/max of contact force + area
    fc = cp_stats['total'][..., -N:]
    ac = ca_stats['total'][..., -N:]
    pc = fc/np.maximum(ac, np.finfo(float).eps)
    t = t[..., -N:]

    # Closed quotient
    cq = np.sum(pc > 0, axis=-1) / N

    # Average contact pressure (during contact)
    tsavg_pc_entire = np.trapz(pc, x=t, axis=-1) / (t[..., -1]-t[..., 0])
    tsavg_pc_contact = tsavg_pc_entire * 1/cq
    return tsavg_pc_contact

# TODO: Make this more general
def plot_contact_vcov_trends(
        axs, vcovs, stats_contact_p, stats_contact_a, ts, fund_freqs,
        add_secondary_axis=False, **plot_kwargs
    ):
    """
    Plot a summary of contact stats given arrays

    Plots a set of contact related statistics as function of swelling.

    Parameters
    ----------
    axs :
        A numpy object array of axes. One statistic is plotted in each axis
    vcovs : ndarray, (m,)
        An array with `m` swelling points
    stats_contact_p, stats_contact_a : ndarray, (m, n)
        An array with `m` swelling points and `n` time points
    """
    N = stats_contact_p.shape[1]//2
    # Load total/max of contact force + area
    smax_pc = stats_contact_p['max'][:, -N:]
    fc = stats_contact_p['total'][:, -N:]

    smax_pa = stats_contact_a['max'][:, -N:]
    ac = stats_contact_a['total'][:, -N:]

    _ac = ac.copy()
    _ac[fc <= 0] = np.inf
    savg_pc = fc/_ac

    t = ts[:, -N:]
    fund_freq = fund_freqs
    period = 1/fund_freq
    # print(savg_pc.shape)

    # Closed quotient
    cq = np.sum(savg_pc > 0, axis=-1) / N
    # axs[2].plot(vcovs, cq, **plot_kwargs)

    # Contact impulse
    # print(ttotal.shape)
    ttotal = t[:, -1] - t[:,0]
    # jc = np.trapz(fc, x=t, axis=-1)
    # # axs[1].plot(vcovs, jc, marker='.', label=f"$m'={mcov:.1f}$")
    # axs[1].plot(vcovs, jc*(period/ttotal), **plot_kwargs)

    # Average contact pressure (during contact)
    tsavg_pc_entire = np.trapz(savg_pc, x=t, axis=-1) / ttotal
    tsavg_pc_contact = tsavg_pc_entire * 1/cq
    axs[0].plot(vcovs, tsavg_pc_contact/10/1e3, **plot_kwargs)
    axs[0].set_ylabel(r"$\hat{p}_\mathrm{c}$ [\si{\kPa}]")

    tsmax_pc_entire = np.max(smax_pc, axis=-1)
    axs[1].plot(vcovs, tsmax_pc_entire/10/1e3, **plot_kwargs)
    axs[1].set_ylabel(r"$\underset{s, t}{\max} \, p_\mathrm{c}$ [\si{\kPa}]")

    # Contact area
    carea_entire = np.sum(ac, axis=-1) / N
    carea_contact = carea_entire * 1/cq
    axs[2].plot(vcovs, carea_contact, **plot_kwargs)
    axs[2].set_ylabel(r"$\tilde{A}_\mathrm{c}$ [\si{\cm}]")

    # Create secondary axis
    if add_secondary_axis:
        _y0s = [
            (tsavg_pc_contact/10/1e3)[0],
            (tsmax_pc_entire/10/1e3)[0],
            carea_contact[0]
        ]
        for ax, y0 in zip(axs, _y0s):
            ax.secondary_yaxis(
                location='right',
                ylabel='$[\si{\percent}]$',
                functions=(
                    lambda x, y0=y0: per_change_forward(x, y0),
                    lambda x, y0=y0: per_change_inv(x, y0)
                )
            )

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(3, 1, figsize=SIZE_LARGE, sharex=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, FIG_LY), sharex=True)

for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
    kwargs_line = {
        'ls': ls,
        'color': color
    }
    t = ts[mcov]
    # f0 = fund_freqs[mcov]
    f0 = 1

    if mcov == -0.8:
        add_secondary_axis = True
    else:
        add_secondary_axis = False
    plot_contact_vcov_trends(
        axs, vcovs, stats_contact_p[mcov], stats_contact_a[mcov], t, f0,
        label=f"$\\bar{{m}}'={mcov:.1f}$",
        add_secondary_axis=add_secondary_axis,
        **kwargs_line
    )

for ax in axs:
    ax.set_xticks([1, 1.15, 1.3])

# for label, ax in zip('abcd', axs.flat):
#     ax.text(0.05, 0.95, f"{label})", transform=ax.transAxes, va='top')
axs[-1].set_xlabel("$v$")

# axs[0].legend(loc='lower left', bbox_to_anchor=(0, 1))

axs[0].legend(loc='lower left', ncol=2, bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/ContactPressureVsV.{FIG_EXT}", dpi=DPI)

### Cover stress vs $v$

In [ ]:
# Process the trend of contact pressure vs v for each mcov case
stress_field_type = 'field_tavg_vm'
# stress_field_type = 'fp_stress_vm'
# stress_field_type = 'ini_stress_vm'

mcovs = np.array([0, -0.8, -1.6])
mcovs = MCOVS

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(1, 1, figsize=SIZE_SMALL, sharex=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 1, figsize=(FIG_LX_WIDE/2, FIG_LY), sharex=True)
axs = np.atleast_1d(axs)

# mcovs = np.array([0, -0.4, -0.8, -1.2, -1.6])

stat_types = ['avg', 'max']
for ax, stat_type in zip(axs, stat_types):
    if stat_type == 'max':
        region = 'medial'
    else:
        region = 'cover'
    summary_type = f'{region}_{stat_type}'
    param_dicts = [{'vcov': vcov} for vcov in vcovs]
    data = {
        mcov: load_measures_np(
            DATA, f'{summary_type}_{stress_field_type}',
            param_dicts,
            {**default_param, 'mcov': mcov}
        )
        for mcov in mcovs
    }

    for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
        kwargs_line = {
            'ls': ls,
            'color': color
        }
        ys = data[mcov]
        ax.plot(VCOVS, ys/10/1e3, **kwargs_line, label=f"$\\bar{{m}}'={mcov:.1f}$")

    y0 = data[-0.8][0]/10/1e3
    ax.secondary_yaxis(
        location='right',
        ylabel='$[\si{\percent}]$',
        functions=(
            lambda x, y0=y0: per_change_forward(x, y0),
            lambda x, y0=y0: per_change_inv(x, y0)
        )
    )

# if FIG_TYPE == 'manuscript':
#     legend_lines = (
#         [mpl.lines.Line2D([0], [0], color=color, ls=ls) for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE)]
#         + [mpl.lines.Line2D([0], [0], ls='-', color='k'), mpl.lines.Line2D([0], [0], ls=':', color='k')]
#     )
#     legend_labels = [f"$m'={mcov:.1f}$" for mcov in mcovs] #+ region_types
# elif FIG_TYPE == 'presentation':
#     legend_lines = (
#         [mpl.lines.Line2D([0], [0], color=color, ls=ls) for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE)]
#         + [mpl.lines.Line2D([0], [0], ls='-', color='k'), mpl.lines.Line2D([0], [0], ls=':', color='k')]
#     )
#     legend_labels = [f"$m'={mcov:.1f}$" for mcov in mcovs]
# axs[0].legend(legend_lines, legend_labels, loc=(0, 1.05))

# axs[0].legend(loc=(0, 1.05))
axs[0].legend(loc='lower left', ncol=2, bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')

for ax in axs.flat:
    ax.set_xlabel("$v$")

ylabels = [
    r"$\hat{\sigma}_\mathrm{vm}$ [kPa]",
    r"$\hat{\sigma}_\mathrm{vm}$ [kPa]"
]
for ax, ylabel in zip(axs.flat, ylabels):
    ax.set_ylabel(ylabel)
for ax in axs[1:]:
    axs[0].sharey(ax)
# axs[1].set_ylabel(r"$\mathrm{max}\,{\sigma_\mathrm{vm}}$ [kPa]")

fig.tight_layout()
fig.savefig(f"{FIG_DIR}/AvgVMStressVsV_{stress_field_type}.{FIG_EXT}", dpi=DPI)

### Cover viscous dissipation vs. $v$

In [ ]:
VCOVS = np.array([1.0, 1.05, 1.1, 1.2, 1.25])
MCOVS = np.array([0.0, -0.8])

In [ ]:
default_param = DEFAULT_PARAM.substitute({'ModifyEffect': ''})

param_dicts = [{'vcov': vcov} for vcov in vcovs]
dat = {
    mcov: load_measures_np(
        DATA, 'signal_spatial_stats_viscous',
        param_dicts,
        {**default_param, 'mcov': mcov}
    )
    for mcov in MCOVS
}

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, ax = plt.subplots(1, 1, figsize=(SIZE_SMALL[0], SIZE_SMALL[1]*0.75))
elif FIG_TYPE == 'presentation':
    fig, ax = plt.subplots(1, 1, figsize=(1/2*FIG_LX_WIDE, FIG_LY))

for mcov, ls, color in zip(MCOVS[:], LS_CYCLE, COLOR_CYCLE):
    kwargs_line = {
        'ls': ls,
        'color': color
    }

    N = dat[mcov].shape[-1]
    ys = np.mean(dat[mcov]['avg'][..., -N//2:], axis=-1)

    ax.plot(VCOVS, ys*1e-7/(1e-2**2), label=f"$\\bar{{m}}'={mcov:.1f}$", **kwargs_line)

ys = np.mean(dat[-0.8]['avg'][..., -N//2:], axis=-1)
y0 = ys[0]*1e-7/(1e-2**2)
ax.secondary_yaxis(
    location='right',
    ylabel='$[\si{\percent}]$',
    functions=(
        lambda x, y0=y0: per_change_forward(x, y0),
        lambda x, y0=y0: per_change_inv(x, y0)
    )
)

ax.set_xlabel("$v$")
ax.set_ylabel(r"$\hat{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")

# ax.legend()
ax.legend(loc='lower left', ncol=2, bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/AvgViscRateVsV.{FIG_EXT}", dpi=DPI)

### Average abs Y-Momentum

In [ ]:
DEFAULT_PARAM = DEFAULT_PARAM
DEFAULT_PARAM = DEFAULT_PARAM.substitute({'ModifyEffect': ''})

param_dicts = [{'vcov': vcov} for vcov in vcovs]
dat = {
    mcov: load_measures_np(
        DATA, 'signal_spatial_state_ymom', param_dicts ,
        {**DEFAULT_PARAM, 'mcov': mcov}

    )
    for mcov in MCOVS
}

dat_constmass = {
    mcov: load_measures_np(
        DATA, 'signal_spatial_state_ymom', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov, 'ModifyEffect': 'const_mass'}
    )
    for mcov in MCOVS
}

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL)
elif FIG_TYPE == 'presentation':
    fig, ax = plt.subplots(1, 1, figsize=(1/2*FIG_LX_WIDE, FIG_LY))

for mcov, color in zip(MCOVS[:], COLOR_CYCLE):
    N = dat[mcov].shape[-1]
    # Plot mean (over time) of abs avg (spatial) y-momentum
    # (over the last 0.5s)
    ys = np.mean(np.abs(dat[mcov]['avg'][..., -N//2:]), axis=-1)
    ax.plot(VCOVS, ys*1e-7/(1e-2**2), label=f"$m'={mcov:.1f}$", ls='-', color=color)

    ys = np.mean(np.abs(dat_constmass[mcov]['avg'][..., -N//2:]), axis=-1)
    ax.plot(VCOVS, ys*1e-7/(1e-2**2), label=f"$m'={mcov:.1f}$ const. mass", ls=':', color=color)

ax.set_xlabel("$v$")
ax.set_ylabel(r"y-momentum")

ax.legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/YMomentum.{FIG_EXT}", dpi=DPI)

### Prephonatory gap vs $v$

In [ ]:
DEFAULT_PARAM = DEFAULT_PARAM.substitute({'ModifyEffect': 'const_pregap'})
DEFAULT_PARAM = DEFAULT_PARAM.substitute({'ModifyEffect': 'const_mass'})
DEFAULT_PARAM = DEFAULT_PARAM.substitute({'ModifyEffect': ''})

param_dicts = [{'vcov': vcov} for vcov in VCOVS]
dat = {
    mcov: load_measures_np(
        DATA, 'gw', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in MCOVS
}

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(FIG_LX, 6*CM))

for mcov in MCOVS:
    # Selecting `[:, 0]` gets the initial glottal width, which is the prephonatory gap
    axs[0].plot(VCOVS, dat[mcov][:, 0], label=f"$m'={mcov:.1f}$")

    # Plot the max-min for amplitude
    axs[1].plot(VCOVS, np.max(dat[mcov], axis=-1)-np.min(dat[mcov], axis=-1))

for ax in axs:
    ax.set_xlabel("$v$")

axs[0].set_ylabel("$y_\mathrm{pre}$ [\si{\cm}]")
axs[1].set_ylabel("Amplitude [\si{\cm}]")
axs[0].legend(loc=(0, 1.05))

fig.tight_layout()

fig.savefig(f'{FIG_DIR}/pregap.{FIG_EXT}')

### Damage measures summary

In [ ]:
DEFAULT_PARAM = DEFAULT_PARAM

param_dicts = [{'vcov': vcov} for vcov in VCOVS]
contact_stats_p = {
    mcov: load_measures_np(
        DATA, 'signal_spatial_stats_con_p', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in MCOVS
}
contact_stats_a = {
    mcov: load_measures_np(
        DATA, 'signal_spatial_stats_con_a', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in MCOVS
}
times = {
    mcov: load_measures_np(
        DATA, 'time', param_dicts, {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in MCOVS
}
avg_pc_data = {
    mcov: np.array([
        avg_contact_pressure(t, p, a) for
        t, p, a in zip(times[mcov], contact_stats_p[mcov], contact_stats_a[mcov])
    ])
    for mcov in MCOVS
}
avg_vm_data = {
    mcov: load_measures_np(
        DATA, 'cover_avg_field_tavg_vm', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in MCOVS
}
avg_wvisc_data = {
    mcov: load_measures_np(
        DATA, 'avg_wvisc', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in MCOVS
}

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True)
gs = mpl.gridspec.GridSpec(1, 3, figure=fig)

axs = np.array([fig.add_subplot(gs[0, jj]) for jj in range(gs.ncols)])

ydatas = [avg_wvisc_data, avg_pc_data, avg_vm_data]
ylabels = [
    r"$\hat{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]",
    r"$\hat{p}_\mathrm{c}$ [\si{\kPa}]",
    r"$\hat{\sigma}_\mathrm{vm}$ [kPa]"
]
for ax, ylabel in zip(axs, ylabels):
    ax.set_ylabel(ylabel)
    ax.set_xlabel("$v$")

for ax, ydata in zip(axs, ydatas):
    for mcov, ls, color in zip(MCOVS, LS_CYCLE, COLOR_CYCLE):
        ax.plot(VCOVS, ydata[mcov], label=f"$m'=${mcov:.1f}", ls=ls, color=color)

axs[0].legend(loc='lower left', ncol=2, bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')

n = 4
v = VCOVS[n]
for ax, ydata in zip(axs, ydatas):
    ys = [ydata[mcov][n] for mcov in (0, -1.6)]
    annotate_vertical_trend(ax, "increasing softening", v, *ys)

fig.savefig(f'{FIG_DIR}/DamageMeasureSummary.{FIG_EXT}')

### Compare w/ and w/o specific mechanisms

In [ ]:
PREFIX_GAP = 'const_pregap'
PREFIX_MASS = 'const_mass'
PREFIX_GAPMASS = 'const_mass_pregap'

In [ ]:
default_param = DEFAULT_PARAM.substitute({'mcov': -0.8})
def load_sub_effects(data, measure_name, default_param=default_param):
    ret_data = {}
    ret_data['default'] = load_measures(
        DATA, measure_name,
        [{'vcov': vcov, 'ModifyEffect': ''} for vcov in VCOVS],
        default_param
    )

    ret_data['const_stiff'] = load_measures(
        DATA, measure_name,
        [{'vcov': vcov, 'mcov': 0.0, 'ModifyEffect': ''} for vcov in VCOVS],
        default_param
    )

    ret_data['const_mass'] = load_measures(
        DATA, measure_name,
        [{'vcov': vcov, 'ModifyEffect': PREFIX_MASS} for vcov in VCOVS],
        default_param
    )

    ret_data['const_pregap'] = load_measures(
        DATA, measure_name,
        [{'vcov': vcov, 'ModifyEffect': PREFIX_GAP} for vcov in VCOVS],
        default_param
    )

    ret_data['const_mass_pregap_stiffness'] = load_measures(
        DATA, measure_name,
        [{'vcov': vcov, 'ModifyEffect': PREFIX_GAPMASS, 'mcov': 0} for vcov in VCOVS],
        default_param
    )
    return ret_data

effectname_to_ylabel = {
    'default': "all swelling effects",
    'const_stiff': "no stiffness change",
    'const_mass': "no mass change",
    'const_pregap': "no gap change",
    'const_mass_pregap_stiffness': "no stiffness, mass, or gap change"
}

#### Subeffects summary

In [ ]:
times = load_sub_effects(DATA, 'time')
contact_stats_p = load_sub_effects(DATA, 'signal_spatial_stats_con_p')
contact_stats_a = load_sub_effects(DATA, 'signal_spatial_stats_con_a')

contact_p = {
    key: [
        avg_contact_pressure(t, cp, ca)
        for t, cp, ca in zip(times[key], contact_stats_p[key], contact_stats_a[key])
    ]
    for key in contact_stats_p.keys()
}

avg_vm_data = load_sub_effects(DATA, 'cover_avg_field_tavg_vm')

avg_wvisc_data = load_sub_effects(DATA, 'avg_wvisc')

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=SIZE_LARGE, sharex=True)

ax = axs[0]
for (effect_name, ys), ls in zip(avg_vm_data.items(), LS_CYCLE):
    ax.plot(VCOVS, np.array(ys)/10/1e3, label=effectname_to_ylabel[effect_name], ls=ls)
ax.set_ylabel(r"$\hat{\sigma}_\mathrm{vm}$ [\si{\kPa}]")

ax = axs[1]
for (effect_name, ys), ls in zip(avg_wvisc_data.items(), LS_CYCLE):
    ax.plot(VCOVS, np.array(ys)*1e-7 / (1e-2)**2, label=effectname_to_ylabel[effect_name], ls=ls)
ax.set_ylabel(r"$\hat{w}_\mathrm{visc}$ $[\si{\W\m^{-2}}]$")

ax = axs[2]
for (effect_name, ys), ls in zip(contact_p.items(), LS_CYCLE):
    ax.plot(VCOVS, np.array(ys)/10/1e3, label=effectname_to_ylabel[effect_name], ls=ls)
ax.set_ylabel(r"$\hat{p}_\mathrm{c}$ [\si{\kPa}]")

_y0s = [
    np.array(avg_vm_data['default'])[0]/10/1e3,
    np.array(avg_wvisc_data['default'])[0]*1e-7 / (1e-2)**2,
    np.array(contact_p['default'])[0]/10/1e3
]
for ax, y0 in zip(axs, _y0s):
    ax.secondary_yaxis(
        location='right',
        ylabel='$[\si{\percent}]$',
        functions=(
            lambda x, y0=y0: per_change_forward(x, y0),
            lambda x, y0=y0: per_change_inv(x, y0)
        )
    )

# for label, ax in zip('abcd', axs.flat):
#     ax.text(0.05, 0.95, f"{label})", transform=ax.transAxes, va='top')

# axs[0].legend(loc='lower left', bbox_to_anchor=(0, 1))
axs[0].legend(loc='lower left', bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')

axs[-1].set_xlabel("$v$")
fig.tight_layout()

fig.savefig(f"{FIG_DIR}/SwellingSubeffects.{FIG_EXT}")

#### F0

In [ ]:
# Load the F0 info with gap compensation
data_f0 = load_sub_effects(DATA, 'fund_freq')

In [ ]:
## Compare F0 w/ and w/o gap compensation
fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL, sharey=True)

for mcov in MCOVS[:1]:
    # F0 (w/o gap)
    f0s = data_f0['default']
    ax.plot(VCOVS, f0s, marker='.', label=f"standard")

    # F0 (w/ gap)
    f0s = data_f0['const_pregap']
    ax.plot(VCOVS, f0s, marker='.', label=f"$m'={mcov:.1f}$ const. gap")

    # F0 (w/ gap)
    f0s = data_f0['const_mass']
    ax.plot(VCOVS, f0s, marker='.', label=f"$m'={mcov:.1f}$ const. mass")

ax.set_xlabel("$v$ [1]")
ax.set_ylabel("$F_0$ [Hz]")

ax.legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/F0GapCompensationComparisonVsV.png", dpi=DPI)

#### Contact pressure

In [ ]:
## Load the contact pressure info with gap compensation
prefixes = ['', PREFIX_GAP, PREFIX_MASS, PREFIX_GAPMASS]
mcov = -0.8
default_param = DEFAULT_PARAM.substitute({'mcov': -0.8})
print(DEFAULT_PARAM)

param_dicts = [{'vcov': vcov} for vcov in vcovs]
contact_stats_p = [
    load_measures_np(
        DATA, 'signal_spatial_stats_con_p', param_dicts,
        {**default_param, 'ModifyEffect': prefix}
    )
    for prefix in prefixes
]

contact_stats_a = [
    load_measures_np(
        DATA, 'signal_spatial_stats_con_a', param_dicts,
        {**default_param, 'ModifyEffect': prefix}
    )
    for prefix in prefixes
]

ts = [
    load_measures_np(
        DATA, 'time', param_dicts,
        {**default_param, 'ModifyEffect': prefix}
    )
    for prefix in prefixes
]

fund_freqs = [
    load_measures_np(
        DATA, 'fund_freq', param_dicts,
        {**default_param, 'ModifyEffect': prefix}
    )
    for prefix in prefixes
]

vcovs = VCOVS

In [ ]:
## Compare mean contact pressure w/ and w/o gap compensation
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(3, 1, figsize=(FIG_LX, 0.7*FIG_LY), sharex=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, FIG_LY), sharex=True)

postfixes = ['normal', 'const. gap', 'const. mass', 'const. gap + mass']
for _cp, _ca, _ts, _fund_freqs, postfix in zip(contact_stats_p, contact_stats_a, ts, fund_freqs, postfixes):
    plot_kwargs = {
        'marker': '.',
        # 'label': f"$m'={mcov:.1f}$"+postfix
        'label': postfix
    }
    plot_contact_vcov_trends(axs, vcovs, _cp, _ca, _ts, _fund_freqs, **plot_kwargs)

if FIG_TYPE == 'manuscript':
    ylabels = [
        "$p_\mathrm{c}$ [kPa]\n(over contact)",
        r"$J_\mathrm{c}$ [$1 \times 10^{-5} \mathrm{N s}$]"+"\n(per cycle)"
    ]
elif FIG_TYPE == 'presentation':
    ylabels = [
        "$\mathrm{mean} \, p_\mathrm{c}$ [kPa]\n(over contact)",
        "$\mathrm{max} \, p_\mathrm{c}$ [kPa]",
        r"$A_\mathrm{c}$ [cm]"
    ]
# for ax, ylabel in zip(axs.flat, ylabels):
#     ax.set_ylabel(ylabel)

# for ax in axs[1:]:
#     axs[0].sharey(ax)

for ax in axs:
    ax.set_xlabel("$v$")

axs[0].legend(loc='lower left', bbox_to_anchor=(0, 1))
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/ContactPressureGapCompensationComparisonVsV.{FIG_EXT}")

#### von Mises stress

In [ ]:
## Load the mean von Mises stress data w/ gap compensation
param_dicts = [{'vcov': vcov} for vcov in VCOVS]
data_cover_avg_vm = {
    mcov: load_measures_np(
        DATA, 'cover_avg_field_tini_vm', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in MCOVS[:1]
}
data_cover_avg_vm_gap = {
    mcov: load_measures_np(
        DATA, 'cover_avg_field_tini_vm', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov, 'ModifyEffect': PREFIX_GAP}
    )
    for mcov in MCOVS[:1]
}
data_cover_avg_vm_mass = {
    mcov: load_measures_np(
        DATA, 'cover_avg_field_tini_vm', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov, 'ModifyEffect': PREFIX_MASS}
    )
    for mcov in MCOVS[:1]
}

data_cover_avg_vm = {
    mcov: load_measures_np(
        DATA, 'cover_avg_field_tavg_vm', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov}
    )
    for mcov in MCOVS[:1]
}
data_cover_avg_vm_gap = {
    mcov: load_measures_np(
        DATA, 'cover_avg_field_tavg_vm', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov, 'ModifyEffect': PREFIX_GAP}
    )
    for mcov in MCOVS[:1]
}
data_cover_avg_vm_mass = {
    mcov: load_measures_np(
        DATA, 'cover_avg_field_tavg_vm', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov, 'ModifyEffect': PREFIX_MASS}
    )
    for mcov in MCOVS[:1]
}

In [ ]:
## Compare mean von Mises stress w/ and w/o gap compensation
fig, ax = plt.subplots(1, 1, figsize=SIZE_MED_WIDE, sharey=True)

for mcov in MCOVS[:1]:
    # Avg. cover vm stress (w/o gap)
    vm_stresses = data_cover_avg_vm[mcov]
    ax.plot(VCOVS, vm_stresses/10/1e3, marker='.', label=f"$m'={mcov:.1f}$")

    # Avg. cover vm stress (w/ gap)
    vm_stresses = data_cover_avg_vm_gap[mcov]
    ax.plot(VCOVS, vm_stresses/10/1e3, marker='.', label=f"$m'={mcov:.1f}$ w/ gap")

    # Avg. cover vm stress (w/ mas)
    vm_stresses = data_cover_avg_vm_mass[mcov]
    ax.plot(VCOVS, vm_stresses/10/1e3, marker='.', label=f"$m'={mcov:.1f}$ w/ mass")

ax.set_xlabel("$v$ [1]")

ax.set_ylabel("$\sigma_\mathrm{vm}$ [kPa]")

ax.legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/VMGapCompensationComparisonVsV.{FIG_EXT}", dpi=DPI)

#### Viscous dissipation

#### SPL

In [ ]:
param_dicts = [{'vcov': vcov} for vcov in VCOVS]

prms_trends = {
    mcov: load_measures_np(
        DATA, 'prms', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov})
    for mcov in MCOVS[:1]
}
prms_trends_gap = {
    mcov: load_measures_np(
        DATA, 'prms', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov, 'ModifyEffect': PREFIX_GAP}
    )
    for mcov in MCOVS[:1]
}
prms_trends_mass = {
    mcov: load_measures_np(
        DATA, 'prms', param_dicts,
        {**DEFAULT_PARAM, 'mcov': mcov, 'ModifyEffect': PREFIX_MASS}
    )
    for mcov in MCOVS[:1]
}

In [ ]:
## Compare mean SPL
fig, ax = plt.subplots(1, 1, figsize=SIZE_MED_WIDE, sharey=True)

for mcov in MCOVS[:1]:
    # Avg. cover vm stress (w/o gap)
    y = prms_trends[mcov]
    ax.plot(VCOVS, 20*np.log10(y/20e-6), marker='.', label=f"$m'={mcov:.1f}$")

    # Avg. cover vm stress (w/ gap)
    y = prms_trends_gap[mcov]
    ax.plot(VCOVS, 20*np.log10(y/20e-6), marker='.', label=f"$m'={mcov:.1f}$ w/ gap")

    # Avg. cover vm stress (w/ gap)
    y = prms_trends_mass[mcov]
    ax.plot(VCOVS, 20*np.log10(y/20e-6), marker='.', label=f"$m'={mcov:.1f}$ w/ mass")

ax.set_xlabel("$v$ [1]")
ax.set_ylabel("$SPL$ [dB]")

ax.legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/SPLGapCompensationComparisonVsV.png", dpi=DPI)

### 3D Mesh independence

In [ ]:
default_param = DEFAULT_PARAM.substitute({
    'vcov': 1,
    'mcov': 0,
    'SwellingModel': 'power'
})

clscales = np.array([0.94, 0.75, 0.6, 0.47, 0.38])
nzs = np.array([12, 15, 19, 24, 30])
param_dicts = [
    {'clscale': clscale, 'NZ': nz}
    for clscale, nz in zip(clscales, nzs)
]

wvisc = load_measures_np(
    DATA, 'avg_wvisc', param_dicts, default_param
)
fo = load_measures_np(
    DATA, 'fund_freq', param_dicts, default_param
)
prms = load_measures_np(
    DATA, 'prms', param_dicts, default_param
)

cell_counts = nzs/15 * (0.75/np.array(clscales))**2

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(FIG_LX, FIG_LY), sharex=True)

axs[0].plot(cell_counts, prms)
axs[0].set_ylabel("$p_\mathrm{rms}$")

axs[1].plot(cell_counts, fo)
axs[1].set_ylabel("$f_\mathrm{o}$")

axs[2].plot(cell_counts, wvisc)
axs[2].set_ylabel("$P_\mathrm{visc}$")
axs[2].set_xlabel("$K_\mathrm{cell}$")
axs[2].set_xticks(cell_counts)
axs[2].xaxis.set_tick_params(rotation=-60)

fig.savefig(f'{FIG_DIR}/3DMeshIndependence.svg')

### Mesh and timestep independence

In [ ]:
## Load mesh independence simulation data

# NOTE: For h independence,`dt` can be either 1.25e-5 or 6.25e-6
DEFAULT_PARAM = DEFAULT_PARAM.substitute({
    'MeshName': 'M5_CB_GA3_SplitCover',
    'clscale': 0.5,
    'mcov': -1.6,
    'vcov': 1.3,
    'tf': 0.5
})

signals = {}
# with h5py.File('out_independence/postprocess.h5', mode='r') as f:
with h5py.File('out/postprocess.h5', mode='r') as f:
    h5utils.h5_to_dict(f, signals)

independence_case = 'dt'
independence_case = 'mesh'
independence_case = 'all'
if independence_case == 'mesh':
    dts = [6.25e-6]
    clscales = np.array([2, 1, .5])
elif independence_case == 'dt':
    dts = 5e-5 * 2.0**np.arange(-1, -5, -1)
    clscales = [1]
elif independence_case == 'all':
    dts = 5e-5 * 2.0**np.arange(0, -5, -1)
    clscales = np.array([2, 1, .5])
params_trend = [{'dt': dt, 'clscale': clscale} for dt, clscale in ittls.product(dts, clscales)]
# p_trend = iter_params(params_trend, DEFAULT_PARAM)

measure_names = [
    'signal_spatial_stats_viscous',
    'signal_spatial_stats_con_p',
    'signal_spatial_stats_con_a',
    'signal_spatial_stats_vm'
]
data_measures = {
    key: load_measures(
        signals, key,
        params_trend,
        default_param=DEFAULT_PARAM
    )
    for key in measure_names
}

data_measures['signal_contact_p'] = [
    f['total']/np.maximum(a['total'], np.finfo(float).eps)
    for f, a in zip(
        data_measures['signal_spatial_stats_con_p'],
        data_measures['signal_spatial_stats_con_a']
    )
]

ys = data_measures['signal_spatial_stats_con_a']
cqs = np.array([np.sum(y['total'] > 0)/y['total'].size for y in ys])

In [ ]:
## Plot trends in a selection of measures vs. the refinement factor
# This is done for either mesh or time step refinement (see `independence_case`)

ylabels = {
    'savg_tavg_vm': r"$\hat{\sigma}_\mathrm{vm}$ [\si{\kPa}]",
    'savg_tavg_viscous_rate': r"$\hat{w}_\mathrm{visc}$ [\si{\W\cm^{-2}}]",
    'savg_contact_pressure': r"$\hat{p}_\mathrm{c}$ [\si{\kPa}]",
    'savg_tavg_contact_a': r"$\tilde{A}_\mathrm{c}$ [\si{\cm^2}]",
    'savg_tavg_contact_p': "Avg. contact force [dyne]",
    'savg_tmax_contact_p': "Max. contact force [dyne]",
    'savg_tmax_contact_a': "Max. contact area [cm]",
}
keys = list(ylabels.keys())

data = {}

ys = data_measures['signal_spatial_stats_vm']
data['savg_tavg_vm'] = [np.mean(y['avg'][y.size//2:]) for y in ys]

ys = data_measures['signal_spatial_stats_viscous']
data['savg_tavg_viscous_rate'] = np.array([np.mean(y['avg'][y.size//2:]) for y in ys]) * 1e-7/((1e-2)**2)

ys = data_measures['signal_spatial_stats_con_p']
data['savg_tavg_contact_p'] = [np.mean(y['total'][y.size//2:]) for y in ys]

ys = data_measures['signal_spatial_stats_con_a']
data['savg_tavg_contact_a'] = [np.mean(y['total'][y.size//2:]) for y in ys]

ys = data_measures['signal_spatial_stats_con_p']
data['savg_tmax_contact_p'] = [np.max(y['total'][y.size//2:]) for y in ys]

ys = data_measures['signal_spatial_stats_con_a']
data['savg_tmax_contact_a'] = [np.max(y['total'][y.size//2:]) for y in ys]

ys = data_measures['signal_contact_p']
data['savg_contact_pressure'] = np.array([np.mean(y[y.size//2:]) for y in ys])*1/cqs/10/1e3

In [ ]:
def plot_refinement(
        fig,
        axs,
        xdata,
        ydata,
        zdata,
        ylabel_formatter=None,
        ymarkers=None,
        **plot_kwargs
    ):
    """
    Plot a 3D surface 'z = f(x, y)' as a series of 'z=g(x)' plots for each 'y'

    Parameters
    ----------
    fig : mpl.Figure
    axs : List[mpl.ax.Axes]
    labels : List[str]
    xdata : np.ndarray (n,)
    ydata : np.ndarray (m,)
    zdata : Mapping[str, np.ndarray(n, m)]
    """
    if ymarkers is None:
        ymarkers = ['']*ydata.size

    if ylabel_formatter is None:
        def ylabel_formatter(y):
            return f"{y}"
    for (key, zs), ax in zip(zdata.items(), axs.flat):
        assert zs.shape == (xdata.size, ydata.size)

        for z, y, marker in zip(zs.T, ydata, ymarkers):
            ax.plot(xdata, z, label=ylabel_formatter(y), marker=marker, **plot_kwargs)

#### Time step refinement

In [ ]:
## Plot vs time refinement
if FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(2, 2, sharex=True, figsize=(FIG_LX_WIDE, 3))
else:
    fig, axs = plt.subplots(4, 1, sharex=True, figsize=(FIG_LX, 5))
axs_twin = np.array([ax.twinx() for ax in axs.flat]).reshape(axs.shape)

shape = (len(dts), len(clscales))
zdata = {key: np.reshape(data[key], shape) for key in keys}
zdata_err = {key: 100*np.abs((y-y.flat[-1])/y.flat[-1]) for key, y in zdata.items()}

xs = 5e-5/dts
def ylabel_formatter(clscale):
    return f"Mesh refinement: {1/clscale:.2f}"
plot_refinement(
    fig, axs, 5e-5/dts, clscales, zdata,
    ymarkers=MARKER_CYCLE,
    ylabel_formatter=ylabel_formatter
)
plot_refinement(
    fig, axs_twin, 5e-5/dts, clscales, zdata_err, ls=':',
    ymarkers=MARKER_CYCLE,
    ylabel_formatter=ylabel_formatter
)

for key, ax, ax_twin in zip(keys, axs.flat, axs_twin.flat):
    ax.set_ylabel(ylabels[key])
    ax_twin.set_ylabel("Error $[\si{\percent}]$")

from matplotlib import ticker
for ax, ax_twin in zip(axs.flat, axs_twin.flat):
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
    ax.xaxis.set_minor_formatter(ticker.NullFormatter())
    ax.xaxis.set_tick_params(which='minor', bottom=False)
    ax.set_xticks(xs)
    # ax_twin.set_yscale('log')
    ax_twin.set_ylim(0, 10)

xlabel = "$\Delta t$ refinement factor"
if FIG_TYPE == 'presentation':
    for ax in axs[-1, :]:
        ax.set_xlabel(xlabel)
else:
    axs[-1].set_xlabel(xlabel)

# axs.flat[0].legend(loc=(0, 1))
axs.flat[0].legend(loc='lower left', bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/Independence-{independence_case}.{FIG_EXT}")

#### Mesh refinement

In [ ]:
## Plot vs mesh refinement
fig, axs = plt.subplots(3, 2, sharex=True, figsize=(FIG_LX_WIDE, 10))
axs_twin = np.array([ax.twinx() for ax in axs.flat]).reshape(axs.shape)

shape = (len(dts), len(clscales))
zdata = {key: np.reshape(data[key], shape).T for key in keys}
zdata_err = {key: 100*np.abs((y-y.flat[-1])/y.flat[-1]) for key, y in zdata.items()}

xs = 1/clscales
plot_refinement(fig, axs, xs, dts, zdata)
plot_refinement(fig, axs_twin, xs, dts, zdata_err, ls=':')

for key, ax, ax_twin in zip(keys, axs.flat, axs_twin.flat):
    ax.set_ylabel(ylabels[key])
    ax_twin.set_ylabel("Error [%]")

from matplotlib import ticker
for ax, ax_twin in zip(axs.flat, axs_twin.flat):
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
    ax.xaxis.set_minor_formatter(ticker.NullFormatter())
    ax.xaxis.set_tick_params(which='minor', bottom=False)
    ax.set_xticks(xs)
    # ax_twin.set_ylim(0, 10)

xlabel = "$h$ Refinement factor"
if FIG_TYPE == 'presentation':
    for ax in axs[-1, :]:
        ax.set_xlabel(xlabel)
else:
    axs.flat[-1].set_xlabel(xlabel)

axs.flat[0].legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/Independence-{independence_case}.{FIG_EXT}")

In [ ]:
times = load_measures(
    signals, 'time',
    params_trend,
    default_param=DEFAULT_PARAM
)
gaws = load_measures(
    signals, 'gw',
    params_trend,
    default_param=DEFAULT_PARAM
)

In [ ]:
fig, ax = plt.subplots(1, 1)
for t, gw in zip(times, gaws):
    ax.plot(t, gw)

ax.set_xlabel("Time [s]")
ax.set_ylabel("GAW [cm]")
ax.legend()

### Yang (2017) m' calculation

The below plot's data from Yang et al. - Quantitative Study of the Effects of Dehydration on the Viscoelastic Parameters in the Vocal Fold Mucosa (2017).
The data are from Tables 2 and 3 which measure 1D moduli (stress/strain) during loading and unloading phases, respectively.


In [ ]:
# Assume that the dehydration % in the study represents the percent
# of original water mass lost
dehydrations = np.array([0, .4, .6, .8])

# Calculate changes in volume for an initial unit volume of VF material
initial_water_vol = 0.8
initial_solid_vol = 0.2
water_vols = initial_water_vol * (1-dehydrations)
total_vols = (water_vols+initial_solid_vol)
hydrations = (water_vols)/total_vols
vs = total_vols/total_vols[0]

In [ ]:
# From table 2 (stretch/loading)
mods_stretch = np.array([41374.5, 44846.05, 50198.98, 78157.41])
# From table 3 (release/unloading)
mods_release = np.array([23589.36, 29810.85, 23198.20, 22833.13])
mods_avg = (mods_stretch+mods_release)/2

fig, axs = plt.subplots(3, 1, sharex=True, figsize=(4, 7))

ax = axs[0]
ax.plot(vs, mods_stretch, label='stretch')
ax.plot(vs, mods_release, label='release')
ax.plot(vs, mods_avg, label='avg')
ax.set_ylabel("E [?]")
ax.legend()

mods_stretch_rel = mods_stretch/mods_stretch[0]
mods_release_rel = mods_release/mods_release[0]
mods_avg_rel = mods_avg/mods_avg[0]
ax = axs[1]
ax.plot(vs, mods_stretch_rel, label='stretch')
ax.plot(vs, mods_release_rel, label='release')
ax.plot(vs, mods_avg_rel, label='avg')
ax.set_ylabel("$E/E_{v=1}$")


mslope_stretch = (mods_stretch_rel-1)/(vs-1)
mslope_release = (mods_release_rel-1)/(vs-1)
mslope_avg = (mods_avg_rel-1)/(vs-1)
ax = axs[2]
ax.plot(vs, mslope_stretch, label='stretch')
ax.plot(vs, mslope_release, label='release')
ax.plot(vs, mslope_avg, label='avg')
ax.set_ylabel(r"$m'=\frac{d E/E_{v=1}}{dv}$")
ax.set_yticks(np.linspace(-4, 0, 11))
# ax.set_ylim(-4, 0)
ax.grid()

axs[-1].set_xlabel("v")

# The nominal m' at dehydration level `i` is:
# Note 1 -> 40%, 2 -> 60%, etc.
i = 1
(mods_avg_rel[i]-mods_avg_rel[0])/(vs[i]-vs[0])

## Debug - Linear vs Nonlinear geometry

This is for comparing two swelling models in directories `out_old` and `out_new`.
The old swelling model didn't account for an extra term strain energy term dependent on linear changes in virtual greens-strain. 

In [ ]:
postdir_old = 'out_lin'
postdir_new = 'out_nonlin'
with h5py.File(f'{postdir_old}/postprocess.h5', mode='r') as f:
    data_old = h5utils.h5_to_dict(f, {})

with h5py.File(f'{postdir_new}/postprocess.h5', mode='r') as f:
    data_new = h5utils.h5_to_dict(f, {})

In [ ]:
COLORS = plt.rcParams['axes.prop_cycle'].by_key()['color']
TF = 0.05
PSUB = 300*10
pdefault = DEFAULT_PARAM.substitute({'tf': TF, 'psub': PSUB})

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
mcov = 0.0
vcovs = [1.0]

for vcov, color in zip(vcovs, COLORS):
    idx = slice(None)
    t = load_measures(data_old, 'time', {'vcov': vcov, 'mcov': mcov}, pdefault)
    gw = load_measures(data_old, 'gw', {'vcov': vcov, 'mcov': mcov}, pdefault)
    t, gw = t[idx], gw[idx]
    ax.plot(t, gw, label=f'$v={vcov}$ (old)', ls=':', color=color)

    t = load_measures(data_new, 'time', {'vcov': vcov, 'mcov': mcov}, pdefault)
    gw = load_measures(data_new, 'gw', {'vcov': vcov, 'mcov': mcov}, pdefault)
    t, gw = t[idx], gw[idx]
    ax.plot(t, gw, label=f'$v={vcov}$ (new)', ls='-', color=color)

ax.legend(loc='lower left', bbox_to_anchor=(0, 1))
fig.tight_layout()

# ax.set_xlim(0., 0.002)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Glottal width [cm]")
fig.tight_layout()

# fig.savefig(f'{FIG_DIR}/GlottalWidthVsV.{FIG_EXT}', dpi=200)

## Debug - Volume after swelling


In [ ]:
dis = MODEL.solid.forms['coeff.state.u1']
def_grad = ufl.grad(dis) + ufl.Identity(2)
vol_ratio = ufl.det(def_grad)
vol_ref_cover = dfn.assemble(1*get_dx(MODEL, 'cover'))
vol_def_cover = vol_ratio*get_dx(MODEL, 'cover')

In [ ]:
for params in iter_params({'vcov': [1.0, 1.3], 'mcov': 0.0}):
    # Load the corresponding parameters and appropriate model
    prop = main.set_basic_props(MODEL, params)
    prop['emod_membrane'] = 50e3 * 10
    MODEL.set_prop(prop)

    nload = max(round((params['vcov']-1)/0.025), 1)
    state_fp, _ = main.solve_static_swollen_config(
        MODEL.solid, MODEL.solid.control, MODEL.solid.prop, nload
    )

    state0 = MODEL.state1.copy()
    state0[:] = 0
    state0[['u', 'v', 'a']] = state_fp
    MODEL.set_fin_state(state0)
    # print(vol_ref_cover)
    print(dfn.assemble(vol_def_cover)/vol_ref_cover)

## Debug - Von Mises stress distribution


In [ ]:
mesh = dfn.UnitSquareMesh(40, 40)
W = dfn.FunctionSpace(mesh, 'DG', 0)
V = dfn.VectorFunctionSpace(mesh, 'CG', 1)

dofmap_W = W.dofmap()

u = dfn.Function(V)
v = dfn.TestFunction(V)

emod = dfn.Function(W)
emod.vector()[:] = 1e4
nu = 0.45
# print(mesh.coordinates())

class Bottom(dfn.SubDomain):
    def inside(self, x, on_boundary):
        return dfn.near(x[1], 0.)

class Left(dfn.SubDomain):
    def inside(self, x, on_boundary):
        return dfn.near(x[0], 0.)

class InnerSquare(dfn.SubDomain):
    def inside(self, x, on_boundary):
        in_x = x[0] >= 0.2 and x[0] <= 0.8
        in_y = x[1] >= 0.2 and x[1] <= 0.8
        return in_x and in_y

mf_ds = dfn.MeshFunction('size_t', mesh, 1)
mf_dx = dfn.MeshFunction('size_t', mesh, 2)

dx = dfn.Measure('dx', mesh, subdomain_data=mf_dx)
ds = dfn.Measure('ds', mesh, subdomain_data=mf_ds)
Bottom().mark(mf_ds, 99)
Left().mark(mf_ds, 100)
InnerSquare().mark(mf_dx, 99)

cells_inner = mf_dx.where_equal(99)
dofs_inner = dofmap_W.entity_dofs(mesh, 2, cells_inner)
emod.vector()[dofs_inner] = 10e4

bc_dir = dfn.DirichletBC(V, dfn.Constant([0, 0]), mf_ds, 99)

def form_eps(u):
    eps = 1/2*(ufl.grad(u) + ufl.grad(u).T)
    return ufl.as_tensor(
        [
            [eps[0, 0], eps[0, 1], 0],
            [eps[1, 0], eps[1, 1], 0],
            [        0,         0, 0]
        ]
    )
eps_test = form_eps(v)

eps = form_eps(u)
eps = ufl.as_tensor(
    [
        [eps[0, 0], eps[0, 1], 0],
        [eps[1, 0], eps[1, 1], 0],
        [        0,         0, 0]
    ]
)
lmbda = emod*nu/(1+nu)/(1-2*nu)
mu = emod/2/(1+nu)
sigma = lmbda*ufl.tr(eps)*ufl.Identity(3) + 2*mu*eps

t = dfn.Constant([1, 0])

F = ufl.inner(sigma, eps_test)*dx - ufl.inner(t, v)*ds(100)

sigma_dev = sigma - 1/3*ufl.tr(sigma)*ufl.Identity(3)
sigma_vm = ufl.sqrt(3/2*ufl.inner(sigma_dev, sigma_dev))

dfn.solve(F == 0, u, bc_dir)

In [ ]:
vert_to_vdof = dfn.vertex_to_dof_map(V)
cell_to_sdof = dofmap_W.entity_dofs(mesh, 2)

svm = dfn.project(sigma_vm, W)
stress_zz = dfn.project(sigma[2, 2], W)
# strain_zz = dfn.project(eps[1, 1], W)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_aspect(1.0)
XY = mesh.coordinates()
cells = mesh.cells()

scale = 100
xy = XY + np.array(scale*u.vector()[:])[vert_to_vdof].reshape(-1, 2)
ax.triplot(xy[:, 0], xy[:, 1], cells)

ax.tripcolor(xy[:, 0], xy[:, 1], cells, svm.vector()[cell_to_sdof])